
<h1 id="Electrical-Power-Generation-2">Electrical Power Generation 2<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/electrical_power_generation_1_2/electrical_power2.html#Electrical-Power-Generation-2">¶</a></h1><h2 id="Objective-and-Prerequisites">Objective and Prerequisites<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/electrical_power_generation_1_2/electrical_power2.html#Objective-and-Prerequisites">¶</a></h2><p>This model is an example of an electrical power generation problem (also known as a unit commitment problem). It selects an optimal set of power stations to turn on in order to satisfy anticipated power demand over a 24-hour time horizon.  It is an extension of an earlier 'Electrical Power Generation' example (example 15 from HP Williams); the extension adds the option of using hydroelectric power plants to satisfy demand.</p>
<p>This model is example 16 from the fifth edition of Model Building in Mathematical Programming, by H. Paul Williams on pages 271-272 and 326-327.</p>
<p>This example is of intermediate difficulty; we assume that you know Python and the Gurobi Python API and that you have some knowledge of building mathematical optimization models.</p>
<p><strong>Download the Repository</strong> <br/>
You can download the repository containing this and other examples by clicking <a href="https://github.com/Gurobi/modeling-examples/archive/master.zip">here</a>.</p>
<p><strong>Gurobi License</strong> <br/>
In order to run this Jupyter Notebook properly, you must have a Gurobi license. If you do not have one, you can request an <a href="https://www.gurobi.com/downloads/request-an-evaluation-license/?utm_source=3PW&amp;utm_medium=OT&amp;utm_campaign=WW-MU-EGU-OR-O_LEA-PR_NO-Q3_FY20_WW_JPME_electrical-power-generation_2_COM_EVAL_GITHUB_&amp;utm_term=electrical-power-generation-problem&amp;utm_content=C_JPM">evaluation license</a> as a <em>commercial user</em>, or download a <a href="https://www.gurobi.com/academia/academic-program-and-licenses/?utm_source=3PW&amp;utm_medium=OT&amp;utm_campaign=WW-MU-EGU-OR-O_LEA-PR_NO-Q3_FY20_WW_JPME_electrical-power-generation_2_ACADEMIC_EVAL_GITHUB_&amp;utm_term=electrical-power-generation-problem&amp;utm_content=C_JPM">free license</a> as an <em>academic user</em>.</p>
<hr/>
<h2 id="Problem-Description">Problem Description<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/electrical_power_generation_1_2/electrical_power2.html#Problem-Description">¶</a></h2><p>In this problem, thermal power generation units are grouped into three distinct types, with different characteristics for each type (power output, cost per megawatt hour, startup cost, etc.).  A unit can be on or off, with a startup cost associated with transitioning from off to on, and power output that can fall anywhere between a specified minimum and maximum value when the unit is on.  There are also two hydroelectric plants available, also with different cost and power generation characteristics.  A 24-hour time horizon is divided into 5 discrete time periods, each with an expected total power demand.  The model decides which units to turn on, and when, in order to satisfy demand for each time period.  The model also captures a reserve requirement, where the selected power plants must be capable of increasing their output, while still respecting their maximum output, in order to cope with the situation where actual demand exceeds predicted demand.</p>
<p>A set of generators is available to satisfy power demand for the following day.  Anticipated demand is as follows:</p>
<table>
<thead><tr>
<th>Time Period</th>
<th>Demand (megawatts)</th>
</tr>
</thead>
<tbody>
<tr>
<td>12 pm to 6 am</td>
<td>15000</td>
</tr>
<tr>
<td>6 am to 9 am</td>
<td>30000</td>
</tr>
<tr>
<td>9 am to 3 pm</td>
<td>25000</td>
</tr>
<tr>
<td>3 pm to 6 pm</td>
<td>40000</td>
</tr>
<tr>
<td>6 pm to 12 pm</td>
<td>27000</td>
</tr>
</tbody>
</table>
<p>Thermal generators are grouped into three types, with the following minimum and maximum output for each type (when they are on):</p>
<table>
<thead><tr>
<th>Type</th>
<th>Number available</th>
<th>Minimum output (MW)</th>
<th>Maximum output (MW)</th>
</tr>
</thead>
<tbody>
<tr>
<td>0</td>
<td>12</td>
<td>850</td>
<td>2000</td>
</tr>
<tr>
<td>1</td>
<td>10</td>
<td>1250</td>
<td>1750</td>
</tr>
<tr>
<td>2</td>
<td>5</td>
<td>1500</td>
<td>4000</td>
</tr>
</tbody>
</table>
<p>There are costs associated with using a thermal generator: a cost per hour when the generator is on (and generating its minimum output), a cost per megawatt hour above its minimum, and a startup cost for turning a generator on:</p>
<table>
<thead><tr>
<th>Type</th>
<th>Cost per hour (when on)</th>
<th>Cost per MWh above minimum</th>
<th>Startup cost</th>
</tr>
</thead>
<tbody>
<tr>
<td>0</td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;1000&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-1-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1" style="width: 3.059em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.503em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.43em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-2"><span class="mi" id="MathJax-Span-3" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-4" style="font-family: MathJax_Main;">1000</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>1000</mn></math></span></span><script id="MathJax-Element-1" type="math/tex">\$1000</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;2.00&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-2-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-5" style="width: 2.781em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.295em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.23em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-6"><span class="mi" id="MathJax-Span-7" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-8" style="font-family: MathJax_Main;">2.00</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>2.00</mn></math></span></span><script id="MathJax-Element-2" type="math/tex">\$2.00</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;2000&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-3-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-9" style="width: 3.059em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.503em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.43em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-10"><span class="mi" id="MathJax-Span-11" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-12" style="font-family: MathJax_Main;">2000</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>2000</mn></math></span></span><script id="MathJax-Element-3" type="math/tex">\$2000</script></td>
</tr>
<tr>
<td>1</td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;2600&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-4-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-13" style="width: 3.059em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.503em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.43em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-14"><span class="mi" id="MathJax-Span-15" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-16" style="font-family: MathJax_Main;">2600</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>2600</mn></math></span></span><script id="MathJax-Element-4" type="math/tex">\$2600</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;1.30&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-5-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-17" style="width: 2.781em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.295em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.23em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-18"><span class="mi" id="MathJax-Span-19" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-20" style="font-family: MathJax_Main;">1.30</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>1.30</mn></math></span></span><script id="MathJax-Element-5" type="math/tex">\$1.30</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;1000&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-6-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-21" style="width: 3.059em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.503em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.43em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-22"><span class="mi" id="MathJax-Span-23" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-24" style="font-family: MathJax_Main;">1000</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>1000</mn></math></span></span><script id="MathJax-Element-6" type="math/tex">\$1000</script></td>
</tr>
<tr>
<td>2</td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;3000&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-7-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-25" style="width: 3.059em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.503em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.43em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-26"><span class="mi" id="MathJax-Span-27" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-28" style="font-family: MathJax_Main;">3000</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>3000</mn></math></span></span><script id="MathJax-Element-7" type="math/tex">\$3000</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;3.00&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-8-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-29" style="width: 2.781em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.295em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.23em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-30"><span class="mi" id="MathJax-Span-31" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-32" style="font-family: MathJax_Main;">3.00</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>3.00</mn></math></span></span><script id="MathJax-Element-8" type="math/tex">\$3.00</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;500&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-9-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-33" style="width: 2.434em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.017em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1001.95em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-34"><span class="mi" id="MathJax-Span-35" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-36" style="font-family: MathJax_Main;">500</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>500</mn></math></span></span><script id="MathJax-Element-9" type="math/tex">\$500</script></td>
</tr>
</tbody>
</table>
<p>Two hydroelectric generators are also available, each with a fixed power output (when on):</p>
<table>
<thead><tr>
<th>Hydro plant</th>
<th>Output (MW)</th>
</tr>
</thead>
<tbody>
<tr>
<td>A</td>
<td>900</td>
</tr>
<tr>
<td>B</td>
<td>1400</td>
</tr>
</tbody>
</table>
<p>The costs associated with using a hydro plant are slightly different.  There's an hourly cost, but it is much smaller than the hourly cost of a thermal generator.  The real cost for a hydroelectric plant comes from depletion of the water in the reservoir, which happens at different rates for the two units.  The reservoir must be replenished before the end of the time horizon by pumping water into it, which consumes electricity.  A hydroelectric plant also has a startup cost.</p>
<table>
<thead><tr>
<th>Hydro plant</th>
<th>Cost per hour (when on)</th>
<th>Startup cost</th>
<th>Reservoir depth reduction (m/hr)</th>
</tr>
</thead>
<tbody>
<tr>
<td>A</td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;90&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-10-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-37" style="width: 1.878em; display: inline-block;"><span style="display: inline-block; position: relative; width: 1.531em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1001.46em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-38"><span class="mi" id="MathJax-Span-39" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-40" style="font-family: MathJax_Main;">90</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>90</mn></math></span></span><script id="MathJax-Element-10" type="math/tex">\$90</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;1500&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-11-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-41" style="width: 3.059em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.503em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.43em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-42"><span class="mi" id="MathJax-Span-43" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-44" style="font-family: MathJax_Main;">1500</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>1500</mn></math></span></span><script id="MathJax-Element-11" type="math/tex">\$1500</script></td>
<td>0.31</td>
</tr>
<tr>
<td>B</td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;150&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-12-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-45" style="width: 2.434em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.017em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1001.95em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-46"><span class="mi" id="MathJax-Span-47" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-48" style="font-family: MathJax_Main;">150</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>150</mn></math></span></span><script id="MathJax-Element-12" type="math/tex">\$150</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;1200&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-13-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-49" style="width: 3.059em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.503em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.43em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-50"><span class="mi" id="MathJax-Span-51" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-52" style="font-family: MathJax_Main;">1200</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>1200</mn></math></span></span><script id="MathJax-Element-13" type="math/tex">\$1200</script></td>
<td>0.47</td>
</tr>
</tbody>
</table>
<p>Pumping water into the reservoir consumes electricity at a rate of 3000 MWh of electricity per meter of height.  The height of the reservoir at the end of the time horizon must be equal to the height at the beginning.</p>
<p>Generators must meet predicted demand, but they must also have sufficient reserve capacity to be able to cope with the situation where actual demand exceeds predicted demand.  For this model, the set of selected thermal generators plus the set of hydro generators must be able to produce as much as 115% of predicted demand.</p>
<p>Which generators should be committed to meet anticipated demand in order to minimize total cost?</p>
<hr/>
<h2 id="Model-Formulation">Model Formulation<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/electrical_power_generation_1_2/electrical_power2.html#Model-Formulation">¶</a></h2><h3 id="Sets-and-Indices">Sets and Indices<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/electrical_power_generation_1_2/electrical_power2.html#Sets-and-Indices">¶</a></h3><p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Types&lt;/mtext&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;mo fence="false" stretchy="false"&gt;{&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;mo fence="false" stretchy="false"&gt;}&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-14-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-53" style="width: 10.896em; display: inline-block;"><span style="display: inline-block; position: relative; width: 9.051em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1008.99em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-54"><span class="mi" id="MathJax-Span-55" style="font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-56" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-57" style="font-family: MathJax_Main; padding-left: 0.301em;">Types</span><span class="mo" id="MathJax-Span-58" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="mo" id="MathJax-Span-59" style="font-family: MathJax_Main; padding-left: 0.301em;">{</span><span class="mn" id="MathJax-Span-60" style="font-family: MathJax_Main;">0</span><span class="mo" id="MathJax-Span-61" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-62" style="font-family: MathJax_Main; padding-left: 0.182em;">1</span><span class="mo" id="MathJax-Span-63" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-64" style="font-family: MathJax_Main; padding-left: 0.182em;">2</span><span class="mo" id="MathJax-Span-65" style="font-family: MathJax_Main;">}</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.361em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>t</mi><mo>∈</mo><mtext>Types</mtext><mo>=</mo><mo fence="false" stretchy="false">{</mo><mn>0</mn><mo>,</mo><mn>1</mn><mo>,</mo><mn>2</mn><mo fence="false" stretchy="false">}</mo></math></span></span><script id="MathJax-Element-14" type="math/tex">t \in \text{Types}=\{0,1,2\}</script>: Types of thermal generators.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;HydroUnits&lt;/mtext&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;mo fence="false" stretchy="false"&gt;{&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo fence="false" stretchy="false"&gt;}&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-15-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-66" style="width: 12.979em; display: inline-block;"><span style="display: inline-block; position: relative; width: 10.777em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1010.72em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-67"><span class="mi" id="MathJax-Span-68" style="font-family: MathJax_Math-italic;">h</span><span class="mo" id="MathJax-Span-69" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-70" style="font-family: MathJax_Main; padding-left: 0.301em;">HydroUnits</span><span class="mo" id="MathJax-Span-71" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="mo" id="MathJax-Span-72" style="font-family: MathJax_Main; padding-left: 0.301em;">{</span><span class="mn" id="MathJax-Span-73" style="font-family: MathJax_Main;">0</span><span class="mo" id="MathJax-Span-74" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-75" style="font-family: MathJax_Main; padding-left: 0.182em;">1</span><span class="mo" id="MathJax-Span-76" style="font-family: MathJax_Main;">}</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.361em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>h</mi><mo>∈</mo><mtext>HydroUnits</mtext><mo>=</mo><mo fence="false" stretchy="false">{</mo><mn>0</mn><mo>,</mo><mn>1</mn><mo fence="false" stretchy="false">}</mo></math></span></span><script id="MathJax-Element-15" type="math/tex">h \in \text{HydroUnits}=\{0,1\}</script>: Two hydro generators.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Periods&lt;/mtext&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;mo fence="false" stretchy="false"&gt;{&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;3&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;4&lt;/mn&gt;&lt;mo fence="false" stretchy="false"&gt;}&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-16-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-77" style="width: 14.11em; display: inline-block;"><span style="display: inline-block; position: relative; width: 11.729em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1011.67em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-78"><span class="mi" id="MathJax-Span-79" style="font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-80" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-81" style="font-family: MathJax_Main; padding-left: 0.301em;">Periods</span><span class="mo" id="MathJax-Span-82" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="mo" id="MathJax-Span-83" style="font-family: MathJax_Main; padding-left: 0.301em;">{</span><span class="mn" id="MathJax-Span-84" style="font-family: MathJax_Main;">0</span><span class="mo" id="MathJax-Span-85" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-86" style="font-family: MathJax_Main; padding-left: 0.182em;">1</span><span class="mo" id="MathJax-Span-87" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-88" style="font-family: MathJax_Main; padding-left: 0.182em;">2</span><span class="mo" id="MathJax-Span-89" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-90" style="font-family: MathJax_Main; padding-left: 0.182em;">3</span><span class="mo" id="MathJax-Span-91" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-92" style="font-family: MathJax_Main; padding-left: 0.182em;">4</span><span class="mo" id="MathJax-Span-93" style="font-family: MathJax_Main;">}</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.361em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>p</mi><mo>∈</mo><mtext>Periods</mtext><mo>=</mo><mo fence="false" stretchy="false">{</mo><mn>0</mn><mo>,</mo><mn>1</mn><mo>,</mo><mn>2</mn><mo>,</mo><mn>3</mn><mo>,</mo><mn>4</mn><mo fence="false" stretchy="false">}</mo></math></span></span><script id="MathJax-Element-16" type="math/tex">p \in \text{Periods}=\{0,1,2,3,4\}</script>: Time periods.</p>
<h3 id="Parameters">Parameters<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/electrical_power_generation_1_2/electrical_power2.html#Parameters">¶</a></h3><p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;period_hours&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;N&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-17-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-94" style="width: 10.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 8.693em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1008.69em, 2.741em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-95"><span class="msubsup" id="MathJax-Span-96"><span style="display: inline-block; position: relative; width: 6.074em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1005.6em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-97" style="font-family: MathJax_Main;">period_hours</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 5.598em;"><span class="texatom" id="MathJax-Span-98"><span class="mrow" id="MathJax-Span-99"><span class="mi" id="MathJax-Span-100" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-101" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-102" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-103"><span class="mrow" id="MathJax-Span-104"><span class="mi" id="MathJax-Span-105" style="font-family: MathJax_AMS;">N</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-106" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.496em; border-left: 0px solid; width: 0px; height: 1.575em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>period_hours</mtext><mrow class="MJX-TeXAtom-ORD"><mi>p</mi></mrow></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">N</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-17" type="math/tex">\text{period_hours}_{p} \in \mathbb{N}^+</script>: Number of hours in each time period.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;demand&lt;/mtext&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-18-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-107" style="width: 7.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 6.491em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1006.49em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-108"><span class="msubsup" id="MathJax-Span-109"><span style="display: inline-block; position: relative; width: 3.872em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.46em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-110" style="font-family: MathJax_Main;">demand</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 3.455em;"><span class="mi" id="MathJax-Span-111" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-112" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-113" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-114"><span class="mrow" id="MathJax-Span-115"><span class="mi" id="MathJax-Span-116" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-117" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.504em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>demand</mtext><mi>p</mi></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-18" type="math/tex">\text{demand}_p \in \mathbb{R}^+</script>: Total power demand for time period <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-19-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-118" style="width: 0.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.479em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.48em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-119"><span class="mi" id="MathJax-Span-120" style="font-family: MathJax_Math-italic;">p</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>p</mi></math></span></span><script id="MathJax-Element-19" type="math/tex">p</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;generators&lt;/mtext&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;N&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-20-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-121" style="width: 8.932em; display: inline-block;"><span style="display: inline-block; position: relative; width: 7.443em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1007.44em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-122"><span class="msubsup" id="MathJax-Span-123"><span style="display: inline-block; position: relative; width: 4.824em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1004.47em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-124" style="font-family: MathJax_Main;">generators</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 4.527em;"><span class="mi" id="MathJax-Span-125" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-126" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-127" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-128"><span class="mrow" id="MathJax-Span-129"><span class="mi" id="MathJax-Span-130" style="font-family: MathJax_AMS;">N</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-131" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>generators</mtext><mi>t</mi></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">N</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-20" type="math/tex">\text{generators}_t \in \mathbb{N}^+</script>: Number of thermal generators of type <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-21-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-132" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-133"><span class="mi" id="MathJax-Span-134" style="font-family: MathJax_Math-italic;">t</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>t</mi></math></span></span><script id="MathJax-Element-21" type="math/tex">t</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mtext&gt;start0&lt;/mtext&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;N&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-22-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-135" style="width: 6.253em; display: inline-block;"><span style="display: inline-block; position: relative; width: 5.182em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1005.18em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-136"><span class="mtext" id="MathJax-Span-137" style="font-family: MathJax_Main;">start0</span><span class="mo" id="MathJax-Span-138" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-139" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-140"><span class="mrow" id="MathJax-Span-141"><span class="mi" id="MathJax-Span-142" style="font-family: MathJax_AMS;">N</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-143" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.146em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mtext>start0</mtext><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">N</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-22" type="math/tex">\text{start0} \in \mathbb{N}^+</script>: Number of thermal generators that are on at the beginning of the time horizon (and available in time period 0 without paying a startup cost).</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;min_output&lt;/mtext&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-23-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-144" style="width: 9.646em; display: inline-block;"><span style="display: inline-block; position: relative; width: 8.039em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1008.04em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-145"><span class="msubsup" id="MathJax-Span-146"><span style="display: inline-block; position: relative; width: 5.42em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1005.06em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-147" style="font-family: MathJax_Main;">min_output</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 5.122em;"><span class="mi" id="MathJax-Span-148" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-149" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-150" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-151"><span class="mrow" id="MathJax-Span-152"><span class="mi" id="MathJax-Span-153" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-154" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>min_output</mtext><mi>t</mi></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-23" type="math/tex">\text{min_output}_t \in \mathbb{R}^+</script>: Minimum output for thermal generator type <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-24-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-155" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-156"><span class="mi" id="MathJax-Span-157" style="font-family: MathJax_Math-italic;">t</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>t</mi></math></span></span><script id="MathJax-Element-24" type="math/tex">t</script> (when on).</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;max_output&lt;/mtext&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-25-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-158" style="width: 9.943em; display: inline-block;"><span style="display: inline-block; position: relative; width: 8.277em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1008.28em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-159"><span class="msubsup" id="MathJax-Span-160"><span style="display: inline-block; position: relative; width: 5.658em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1005.24em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-161" style="font-family: MathJax_Main;">max_output</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 5.301em;"><span class="mi" id="MathJax-Span-162" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-163" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-164" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-165"><span class="mrow" id="MathJax-Span-166"><span class="mi" id="MathJax-Span-167" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-168" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>max_output</mtext><mi>t</mi></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-25" type="math/tex">\text{max_output}_t \in \mathbb{R}^+</script>: Maximum output for thermal generator type <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-26-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-169" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-170"><span class="mi" id="MathJax-Span-171" style="font-family: MathJax_Math-italic;">t</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>t</mi></math></span></span><script id="MathJax-Element-26" type="math/tex">t</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;base_cost&lt;/mtext&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-27-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-172" style="width: 8.515em; display: inline-block;"><span style="display: inline-block; position: relative; width: 7.086em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1007.09em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-173"><span class="msubsup" id="MathJax-Span-174"><span style="display: inline-block; position: relative; width: 4.467em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1004.05em, 4.229em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-175" style="font-family: MathJax_Main;">base_cost</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 4.11em;"><span class="mi" id="MathJax-Span-176" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-177" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-178" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-179"><span class="mrow" id="MathJax-Span-180"><span class="mi" id="MathJax-Span-181" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-182" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.289em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>base_cost</mtext><mi>t</mi></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-27" type="math/tex">\text{base_cost}_t \in \mathbb{R}^+</script>: Minimum operating cost (per hour) for a thermal generator of type <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-28-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-183" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-184"><span class="mi" id="MathJax-Span-185" style="font-family: MathJax_Math-italic;">t</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>t</mi></math></span></span><script id="MathJax-Element-28" type="math/tex">t</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;per_mw_cost&lt;/mtext&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-29-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-186" style="width: 10.36em; display: inline-block;"><span style="display: inline-block; position: relative; width: 8.634em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1008.63em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-187"><span class="msubsup" id="MathJax-Span-188"><span style="display: inline-block; position: relative; width: 6.015em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1005.6em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-189" style="font-family: MathJax_Main;">per_mw_cost</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 5.658em;"><span class="mi" id="MathJax-Span-190" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-191" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-192" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-193"><span class="mrow" id="MathJax-Span-194"><span class="mi" id="MathJax-Span-195" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-196" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>per_mw_cost</mtext><mi>t</mi></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-29" type="math/tex">\text{per_mw_cost}_t \in \mathbb{R}^+</script>: Cost to generate one additional MW (per hour) for a thermal generator of type <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-30-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-197" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-198"><span class="mi" id="MathJax-Span-199" style="font-family: MathJax_Math-italic;">t</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>t</mi></math></span></span><script id="MathJax-Element-30" type="math/tex">t</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;startup_cost&lt;/mtext&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-31-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-200" style="width: 10.003em; display: inline-block;"><span style="display: inline-block; position: relative; width: 8.336em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1008.34em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-201"><span class="msubsup" id="MathJax-Span-202"><span style="display: inline-block; position: relative; width: 5.717em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1005.36em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-203" style="font-family: MathJax_Main;">startup_cost</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 5.42em;"><span class="mi" id="MathJax-Span-204" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-205" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-206" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-207"><span class="mrow" id="MathJax-Span-208"><span class="mi" id="MathJax-Span-209" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-210" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>startup_cost</mtext><mi>t</mi></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-31" type="math/tex">\text{startup_cost}_t \in \mathbb{R}^+</script>: Startup cost for thermal generator of type <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-32-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-211" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-212"><span class="mi" id="MathJax-Span-213" style="font-family: MathJax_Math-italic;">t</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>t</mi></math></span></span><script id="MathJax-Element-32" type="math/tex">t</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;hydro_load&lt;/mtext&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-33-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-214" style="width: 9.586em; display: inline-block;"><span style="display: inline-block; position: relative; width: 7.979em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1007.98em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-215"><span class="msubsup" id="MathJax-Span-216"><span style="display: inline-block; position: relative; width: 5.36em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1004.82em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-217" style="font-family: MathJax_Main;">hydro_load</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 4.884em;"><span class="mi" id="MathJax-Span-218" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-219" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-220" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-221"><span class="mrow" id="MathJax-Span-222"><span class="mi" id="MathJax-Span-223" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-224" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>hydro_load</mtext><mi>h</mi></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-33" type="math/tex">\text{hydro_load}_h \in \mathbb{R}^+</script>: Output for hydro generator <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-34-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-225" style="width: 0.717em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.598em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1000.6em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-226"><span class="mi" id="MathJax-Span-227" style="font-family: MathJax_Math-italic;">h</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>h</mi></math></span></span><script id="MathJax-Element-34" type="math/tex">h</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;hydro_cost&lt;/mtext&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-35-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-228" style="width: 9.467em; display: inline-block;"><span style="display: inline-block; position: relative; width: 7.86em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1007.86em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-229"><span class="msubsup" id="MathJax-Span-230"><span style="display: inline-block; position: relative; width: 5.241em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1004.71em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-231" style="font-family: MathJax_Main;">hydro_cost</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 4.765em;"><span class="mi" id="MathJax-Span-232" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-233" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-234" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-235"><span class="mrow" id="MathJax-Span-236"><span class="mi" id="MathJax-Span-237" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-238" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>hydro_cost</mtext><mi>h</mi></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-35" type="math/tex">\text{hydro_cost}_h \in \mathbb{R}^+</script>: Cost for operating hydro generator <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-36-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-239" style="width: 0.717em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.598em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1000.6em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-240"><span class="mi" id="MathJax-Span-241" style="font-family: MathJax_Math-italic;">h</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>h</mi></math></span></span><script id="MathJax-Element-36" type="math/tex">h</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;hydro_startup_cost&lt;/mtext&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-37-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-242" style="width: 13.872em; display: inline-block;"><span style="display: inline-block; position: relative; width: 11.551em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1011.55em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-243"><span class="msubsup" id="MathJax-Span-244"><span style="display: inline-block; position: relative; width: 8.932em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1008.4em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-245" style="font-family: MathJax_Main;">hydro_startup_cost</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 8.455em;"><span class="mi" id="MathJax-Span-246" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-247" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-248" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-249"><span class="mrow" id="MathJax-Span-250"><span class="mi" id="MathJax-Span-251" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-252" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>hydro_startup_cost</mtext><mi>h</mi></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-37" type="math/tex">\text{hydro_startup_cost}_h \in \mathbb{R}^+</script>: Startup cost for hydro generator <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-38-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-253" style="width: 0.717em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.598em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1000.6em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-254"><span class="mi" id="MathJax-Span-255" style="font-family: MathJax_Math-italic;">h</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>h</mi></math></span></span><script id="MathJax-Element-38" type="math/tex">h</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;hydro_height_reduction&lt;/mtext&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-39-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-256" style="width: 16.193em; display: inline-block;"><span style="display: inline-block; position: relative; width: 13.455em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1013.46em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-257"><span class="msubsup" id="MathJax-Span-258"><span style="display: inline-block; position: relative; width: 10.836em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1010.36em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-259" style="font-family: MathJax_Main;">hydro_height_reduction</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 10.36em;"><span class="mi" id="MathJax-Span-260" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-261" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-262" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-263"><span class="mrow" id="MathJax-Span-264"><span class="mi" id="MathJax-Span-265" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-266" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>hydro_height_reduction</mtext><mi>h</mi></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-39" type="math/tex">\text{hydro_height_reduction}_h \in \mathbb{R}^+</script>: Hourly reduction in reservoir height from operating hydro generator <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-40-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-267" style="width: 0.717em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.598em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1000.6em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-268"><span class="mi" id="MathJax-Span-269" style="font-family: MathJax_Math-italic;">h</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>h</mi></math></span></span><script id="MathJax-Element-40" type="math/tex">h</script>.</p>
<h3 id="Decision-Variables">Decision Variables<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/electrical_power_generation_1_2/electrical_power2.html#Decision-Variables">¶</a></h3><p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;ngen&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;N&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-41-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-270" style="width: 6.67em; display: inline-block;"><span style="display: inline-block; position: relative; width: 5.539em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1005.54em, 2.741em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-271"><span class="msubsup" id="MathJax-Span-272"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1002.03em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-273" style="font-family: MathJax_Main;">ngen</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.086em;"><span class="texatom" id="MathJax-Span-274"><span class="mrow" id="MathJax-Span-275"><span class="mi" id="MathJax-Span-276" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-277" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-278" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-279" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-280" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-281"><span class="mrow" id="MathJax-Span-282"><span class="mi" id="MathJax-Span-283" style="font-family: MathJax_AMS;">N</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-284" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.496em; border-left: 0px solid; width: 0px; height: 1.575em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>ngen</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>p</mi></mrow></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">N</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-41" type="math/tex">\text{ngen}_{t,p} \in \mathbb{N}^+</script>: Number of thermal generators of type <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-42-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-285" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-286"><span class="mi" id="MathJax-Span-287" style="font-family: MathJax_Math-italic;">t</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>t</mi></math></span></span><script id="MathJax-Element-42" type="math/tex">t</script> that are on in time period <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-43-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-288" style="width: 0.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.479em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.48em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-289"><span class="mi" id="MathJax-Span-290" style="font-family: MathJax_Math-italic;">p</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>p</mi></math></span></span><script id="MathJax-Element-43" type="math/tex">p</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;output&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-44-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-291" style="width: 7.741em; display: inline-block;"><span style="display: inline-block; position: relative; width: 6.432em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1006.43em, 2.741em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-292"><span class="msubsup" id="MathJax-Span-293"><span style="display: inline-block; position: relative; width: 3.812em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1002.92em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-294" style="font-family: MathJax_Main;">output</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.92em;"><span class="texatom" id="MathJax-Span-295"><span class="mrow" id="MathJax-Span-296"><span class="mi" id="MathJax-Span-297" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-298" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-299" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-300" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-301" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-302"><span class="mrow" id="MathJax-Span-303"><span class="mi" id="MathJax-Span-304" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-305" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.496em; border-left: 0px solid; width: 0px; height: 1.575em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>output</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>p</mi></mrow></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-44" type="math/tex">\text{output}_{t,p} \in \mathbb{R}^+</script>: Total power output from thermal generators of type <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-45-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-306" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-307"><span class="mi" id="MathJax-Span-308" style="font-family: MathJax_Math-italic;">t</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>t</mi></math></span></span><script id="MathJax-Element-45" type="math/tex">t</script> in time period <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-46-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-309" style="width: 0.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.479em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.48em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-310"><span class="mi" id="MathJax-Span-311" style="font-family: MathJax_Math-italic;">p</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>p</mi></math></span></span><script id="MathJax-Element-46" type="math/tex">p</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;nstart&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;N&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-47-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-312" style="width: 7.384em; display: inline-block;"><span style="display: inline-block; position: relative; width: 6.134em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1006.13em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-313"><span class="msubsup" id="MathJax-Span-314"><span style="display: inline-block; position: relative; width: 3.515em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1002.56em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-315" style="font-family: MathJax_Main;">nstart</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 2.622em;"><span class="texatom" id="MathJax-Span-316"><span class="mrow" id="MathJax-Span-317"><span class="mi" id="MathJax-Span-318" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-319" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-320" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-321" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-322" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-323"><span class="mrow" id="MathJax-Span-324"><span class="mi" id="MathJax-Span-325" style="font-family: MathJax_AMS;">N</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-326" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.504em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>nstart</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>p</mi></mrow></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">N</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-47" type="math/tex">\text{nstart}_{t,p} \in \mathbb{N}^+</script>: Number of thermal generators of type <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-48-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-327" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-328"><span class="mi" id="MathJax-Span-329" style="font-family: MathJax_Math-italic;">t</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>t</mi></math></span></span><script id="MathJax-Element-48" type="math/tex">t</script> to start in time period <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-49-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-330" style="width: 0.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.479em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.48em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-331"><span class="mi" id="MathJax-Span-332" style="font-family: MathJax_Math-italic;">p</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>p</mi></math></span></span><script id="MathJax-Element-49" type="math/tex">p</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;hydro&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mo stretchy="false"&gt;[&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo stretchy="false"&gt;]&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-50-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-333" style="width: 8.217em; display: inline-block;"><span style="display: inline-block; position: relative; width: 6.848em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1006.73em, 2.741em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-334"><span class="msubsup" id="MathJax-Span-335"><span style="display: inline-block; position: relative; width: 3.574em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1002.5em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-336" style="font-family: MathJax_Main;">hydro</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.562em;"><span class="texatom" id="MathJax-Span-337"><span class="mrow" id="MathJax-Span-338"><span class="mi" id="MathJax-Span-339" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span><span class="mo" id="MathJax-Span-340" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-341" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-342" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mo" id="MathJax-Span-343" style="font-family: MathJax_Main; padding-left: 0.301em;">[</span><span class="mn" id="MathJax-Span-344" style="font-family: MathJax_Main;">0</span><span class="mo" id="MathJax-Span-345" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-346" style="font-family: MathJax_Main; padding-left: 0.182em;">1</span><span class="mo" id="MathJax-Span-347" style="font-family: MathJax_Main;">]</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.496em; border-left: 0px solid; width: 0px; height: 1.504em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>hydro</mtext><mrow class="MJX-TeXAtom-ORD"><mi>h</mi><mo>,</mo><mi>p</mi></mrow></msub><mo>∈</mo><mo stretchy="false">[</mo><mn>0</mn><mo>,</mo><mn>1</mn><mo stretchy="false">]</mo></math></span></span><script id="MathJax-Element-50" type="math/tex">\text{hydro}_{h,p} \in [0,1]</script>: Indicates whether hydro generators <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-51-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-348" style="width: 0.717em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.598em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1000.6em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-349"><span class="mi" id="MathJax-Span-350" style="font-family: MathJax_Math-italic;">h</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>h</mi></math></span></span><script id="MathJax-Element-51" type="math/tex">h</script> is on in time period <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-52-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-351" style="width: 0.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.479em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.48em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-352"><span class="mi" id="MathJax-Span-353" style="font-family: MathJax_Math-italic;">p</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>p</mi></math></span></span><script id="MathJax-Element-52" type="math/tex">p</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;hydro_start&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mo stretchy="false"&gt;[&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo stretchy="false"&gt;]&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-53-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-354" style="width: 11.313em; display: inline-block;"><span style="display: inline-block; position: relative; width: 9.408em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1009.29em, 2.741em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-355"><span class="msubsup" id="MathJax-Span-356"><span style="display: inline-block; position: relative; width: 6.134em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1005.06em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-357" style="font-family: MathJax_Main;">hydro_start</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 5.122em;"><span class="texatom" id="MathJax-Span-358"><span class="mrow" id="MathJax-Span-359"><span class="mi" id="MathJax-Span-360" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span><span class="mo" id="MathJax-Span-361" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-362" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-363" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mo" id="MathJax-Span-364" style="font-family: MathJax_Main; padding-left: 0.301em;">[</span><span class="mn" id="MathJax-Span-365" style="font-family: MathJax_Main;">0</span><span class="mo" id="MathJax-Span-366" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-367" style="font-family: MathJax_Main; padding-left: 0.182em;">1</span><span class="mo" id="MathJax-Span-368" style="font-family: MathJax_Main;">]</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.496em; border-left: 0px solid; width: 0px; height: 1.504em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>hydro_start</mtext><mrow class="MJX-TeXAtom-ORD"><mi>h</mi><mo>,</mo><mi>p</mi></mrow></msub><mo>∈</mo><mo stretchy="false">[</mo><mn>0</mn><mo>,</mo><mn>1</mn><mo stretchy="false">]</mo></math></span></span><script id="MathJax-Element-53" type="math/tex">\text{hydro_start}_{h,p} \in [0,1]</script>: Indicates whether hydro generator <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-54-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-369" style="width: 0.717em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.598em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1000.6em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-370"><span class="mi" id="MathJax-Span-371" style="font-family: MathJax_Math-italic;">h</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>h</mi></math></span></span><script id="MathJax-Element-54" type="math/tex">h</script> starts in time period <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-55-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-372" style="width: 0.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.479em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.48em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-373"><span class="mi" id="MathJax-Span-374" style="font-family: MathJax_Math-italic;">p</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>p</mi></math></span></span><script id="MathJax-Element-55" type="math/tex">p</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;height&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-56-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-375" style="width: 6.967em; display: inline-block;"><span style="display: inline-block; position: relative; width: 5.777em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1005.78em, 2.741em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-376"><span class="msubsup" id="MathJax-Span-377"><span style="display: inline-block; position: relative; width: 3.158em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1002.68em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-378" style="font-family: MathJax_Main;">height</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.741em;"><span class="texatom" id="MathJax-Span-379"><span class="mrow" id="MathJax-Span-380"><span class="mi" id="MathJax-Span-381" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-382" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-383" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-384"><span class="mrow" id="MathJax-Span-385"><span class="mi" id="MathJax-Span-386" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-387" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.496em; border-left: 0px solid; width: 0px; height: 1.575em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>height</mtext><mrow class="MJX-TeXAtom-ORD"><mi>p</mi></mrow></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-56" type="math/tex">\text{height}_{p} \in \mathbb{R}^+</script>: Height of reservoir in time period <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-57-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-388" style="width: 0.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.479em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.48em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-389"><span class="mi" id="MathJax-Span-390" style="font-family: MathJax_Math-italic;">p</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>p</mi></math></span></span><script id="MathJax-Element-57" type="math/tex">p</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;pumping&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-58-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-391" style="width: 8.336em; display: inline-block;"><span style="display: inline-block; position: relative; width: 6.908em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1006.91em, 2.741em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-392"><span class="msubsup" id="MathJax-Span-393"><span style="display: inline-block; position: relative; width: 4.289em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1003.81em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-394" style="font-family: MathJax_Main;">pumping</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 3.812em;"><span class="texatom" id="MathJax-Span-395"><span class="mrow" id="MathJax-Span-396"><span class="mi" id="MathJax-Span-397" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-398" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-399" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-400"><span class="mrow" id="MathJax-Span-401"><span class="mi" id="MathJax-Span-402" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-403" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.496em; border-left: 0px solid; width: 0px; height: 1.575em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>pumping</mtext><mrow class="MJX-TeXAtom-ORD"><mi>p</mi></mrow></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-58" type="math/tex">\text{pumping}_{p} \in \mathbb{R}^+</script>: Power used to replenish reservoir in time period <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-59-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-404" style="width: 0.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.479em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.48em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-405"><span class="mi" id="MathJax-Span-406" style="font-family: MathJax_Math-italic;">p</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>p</mi></math></span></span><script id="MathJax-Element-59" type="math/tex">p</script>.</p>
<h3 id="Objective-Function">Objective Function<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/electrical_power_generation_1_2/electrical_power2.html#Objective-Function">¶</a></h3><ul>
<li><strong>Cost</strong>: Minimize the cost (in USD) to satisfy the predicted electricity demand.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtext&gt;Minimize&lt;/mtext&gt;&lt;mspace width="1em" /&gt;&lt;msub&gt;&lt;mi&gt;Z&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;o&lt;/mi&gt;&lt;mi&gt;n&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;Z&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;e&lt;/mi&gt;&lt;mi&gt;x&lt;/mi&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mi&gt;r&lt;/mi&gt;&lt;mi&gt;a&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;Z&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mi&gt;a&lt;/mi&gt;&lt;mi&gt;r&lt;/mi&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mi&gt;u&lt;/mi&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;Z&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;mi&gt;r&lt;/mi&gt;&lt;mi&gt;o&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;Z&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;mi&gt;r&lt;/mi&gt;&lt;mi&gt;o&lt;/mi&gt;&lt;mi mathvariant="normal"&gt;&amp;#x005F;&lt;/mi&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mi&gt;a&lt;/mi&gt;&lt;mi&gt;r&lt;/mi&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mi&gt;u&lt;/mi&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/math&gt;' id="MathJax-Element-60-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-407" style="width: 29.884em; display: inline-block;"><span style="display: inline-block; position: relative; width: 24.884em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1024.88em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-408"><span class="mtext" id="MathJax-Span-409" style="font-family: MathJax_Main;">Minimize</span><span class="mspace" id="MathJax-Span-410" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="msubsup" id="MathJax-Span-411"><span style="display: inline-block; position: relative; width: 1.551em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-412" style="font-family: MathJax_Math-italic;">Z<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.658em;"><span class="texatom" id="MathJax-Span-413"><span class="mrow" id="MathJax-Span-414"><span class="mi" id="MathJax-Span-415" style="font-size: 70.7%; font-family: MathJax_Math-italic;">o</span><span class="mi" id="MathJax-Span-416" style="font-size: 70.7%; font-family: MathJax_Math-italic;">n</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-417" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="msubsup" id="MathJax-Span-418" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.443em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-419" style="font-family: MathJax_Math-italic;">Z<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.658em;"><span class="texatom" id="MathJax-Span-420"><span class="mrow" id="MathJax-Span-421"><span class="mi" id="MathJax-Span-422" style="font-size: 70.7%; font-family: MathJax_Math-italic;">e</span><span class="mi" id="MathJax-Span-423" style="font-size: 70.7%; font-family: MathJax_Math-italic;">x</span><span class="mi" id="MathJax-Span-424" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mi" id="MathJax-Span-425" style="font-size: 70.7%; font-family: MathJax_Math-italic;">r</span><span class="mi" id="MathJax-Span-426" style="font-size: 70.7%; font-family: MathJax_Math-italic;">a</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-427" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="msubsup" id="MathJax-Span-428" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 3.039em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-429" style="font-family: MathJax_Math-italic;">Z<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.658em;"><span class="texatom" id="MathJax-Span-430"><span class="mrow" id="MathJax-Span-431"><span class="mi" id="MathJax-Span-432" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mi" id="MathJax-Span-433" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mi" id="MathJax-Span-434" style="font-size: 70.7%; font-family: MathJax_Math-italic;">a</span><span class="mi" id="MathJax-Span-435" style="font-size: 70.7%; font-family: MathJax_Math-italic;">r</span><span class="mi" id="MathJax-Span-436" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mi" id="MathJax-Span-437" style="font-size: 70.7%; font-family: MathJax_Math-italic;">u</span><span class="mi" id="MathJax-Span-438" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-439" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="msubsup" id="MathJax-Span-440" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.562em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-441" style="font-family: MathJax_Math-italic;">Z<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.658em;"><span class="texatom" id="MathJax-Span-442"><span class="mrow" id="MathJax-Span-443"><span class="mi" id="MathJax-Span-444" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span><span class="mi" id="MathJax-Span-445" style="font-size: 70.7%; font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mi" id="MathJax-Span-446" style="font-size: 70.7%; font-family: MathJax_Math-italic;">d<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mi" id="MathJax-Span-447" style="font-size: 70.7%; font-family: MathJax_Math-italic;">r</span><span class="mi" id="MathJax-Span-448" style="font-size: 70.7%; font-family: MathJax_Math-italic;">o</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-449" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="msubsup" id="MathJax-Span-450" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 5.182em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-451" style="font-family: MathJax_Math-italic;">Z<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.658em;"><span class="texatom" id="MathJax-Span-452"><span class="mrow" id="MathJax-Span-453"><span class="mi" id="MathJax-Span-454" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span><span class="mi" id="MathJax-Span-455" style="font-size: 70.7%; font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mi" id="MathJax-Span-456" style="font-size: 70.7%; font-family: MathJax_Math-italic;">d<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mi" id="MathJax-Span-457" style="font-size: 70.7%; font-family: MathJax_Math-italic;">r</span><span class="mi" id="MathJax-Span-458" style="font-size: 70.7%; font-family: MathJax_Math-italic;">o</span><span class="mi" id="MathJax-Span-459" style="font-size: 70.7%; font-family: MathJax_Main;">_</span><span class="mi" id="MathJax-Span-460" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mi" id="MathJax-Span-461" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mi" id="MathJax-Span-462" style="font-size: 70.7%; font-family: MathJax_Math-italic;">a</span><span class="mi" id="MathJax-Span-463" style="font-size: 70.7%; font-family: MathJax_Math-italic;">r</span><span class="mi" id="MathJax-Span-464" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mi" id="MathJax-Span-465" style="font-size: 70.7%; font-family: MathJax_Math-italic;">u</span><span class="mi" id="MathJax-Span-466" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.289em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtext>Minimize</mtext><mspace width="1em"></mspace><msub><mi>Z</mi><mrow class="MJX-TeXAtom-ORD"><mi>o</mi><mi>n</mi></mrow></msub><mo>+</mo><msub><mi>Z</mi><mrow class="MJX-TeXAtom-ORD"><mi>e</mi><mi>x</mi><mi>t</mi><mi>r</mi><mi>a</mi></mrow></msub><mo>+</mo><msub><mi>Z</mi><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mi>t</mi><mi>a</mi><mi>r</mi><mi>t</mi><mi>u</mi><mi>p</mi></mrow></msub><mo>+</mo><msub><mi>Z</mi><mrow class="MJX-TeXAtom-ORD"><mi>h</mi><mi>y</mi><mi>d</mi><mi>r</mi><mi>o</mi></mrow></msub><mo>+</mo><msub><mi>Z</mi><mrow class="MJX-TeXAtom-ORD"><mi>h</mi><mi>y</mi><mi>d</mi><mi>r</mi><mi>o</mi><mi mathvariant="normal">_</mi><mi>s</mi><mi>t</mi><mi>a</mi><mi>r</mi><mi>t</mi><mi>u</mi><mi>p</mi></mrow></msub></math></span></span></div><script id="MathJax-Element-60" type="math/tex; mode=display">\begin{equation}
\text{Minimize} \quad Z_{on} + Z_{extra} + Z_{startup} + Z_{hydro} + Z_{hydro\_startup}
\end{equation}</script><span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;msub&gt;&lt;mi&gt;Z&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;o&lt;/mi&gt;&lt;mi&gt;n&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Types&lt;/mtext&gt;&lt;mo&gt;&amp;#x00D7;&lt;/mo&gt;&lt;mtext&gt;Periods&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;base_cost&lt;/mtext&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;ngen&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;/math&gt;' id="MathJax-Element-61-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-467" style="width: 21.67em; display: inline-block;"><span style="display: inline-block; position: relative; width: 18.039em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.074em, 1018.04em, 3.812em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-468"><span class="msubsup" id="MathJax-Span-469"><span style="display: inline-block; position: relative; width: 1.551em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-470" style="font-family: MathJax_Math-italic;">Z<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.658em;"><span class="texatom" id="MathJax-Span-471"><span class="mrow" id="MathJax-Span-472"><span class="mi" id="MathJax-Span-473" style="font-size: 70.7%; font-family: MathJax_Math-italic;">o</span><span class="mi" id="MathJax-Span-474" style="font-size: 70.7%; font-family: MathJax_Math-italic;">n</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-475" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="munderover" id="MathJax-Span-476" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 6.551em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 2.562em;"><span class="mo" id="MathJax-Span-477" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.277em, 1006.55em, 4.467em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-478"><span class="mrow" id="MathJax-Span-479"><span class="mo" id="MathJax-Span-480" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-481" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-482" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-483" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-484" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-485" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-486" style="font-size: 70.7%; font-family: MathJax_Main;">Types</span><span class="mo" id="MathJax-Span-487" style="font-size: 70.7%; font-family: MathJax_Main;">×</span><span class="mtext" id="MathJax-Span-488" style="font-size: 70.7%; font-family: MathJax_Main;">Periods</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-489" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-490"><span class="msubsup" id="MathJax-Span-491"><span style="display: inline-block; position: relative; width: 4.467em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1004.05em, 4.229em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-492" style="font-family: MathJax_Main;">base_cost</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 4.11em;"><span class="mi" id="MathJax-Span-493" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-494" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="msubsup" id="MathJax-Span-495" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1002.03em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-496" style="font-family: MathJax_Main;">ngen</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.086em;"><span class="texatom" id="MathJax-Span-497"><span class="mrow" id="MathJax-Span-498"><span class="mi" id="MathJax-Span-499" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-500" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-501" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.782em; border-left: 0px solid; width: 0px; height: 3.004em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>Z</mi><mrow class="MJX-TeXAtom-ORD"><mi>o</mi><mi>n</mi></mrow></msub><mo>=</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mo stretchy="false">(</mo><mi>t</mi><mo>,</mo><mi>p</mi><mo stretchy="false">)</mo><mo>∈</mo><mtext>Types</mtext><mo>×</mo><mtext>Periods</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>base_cost</mtext><mi>t</mi></msub><mo>∗</mo><msub><mtext>ngen</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>p</mi></mrow></msub></mrow></math></span></span></div><script id="MathJax-Element-61" type="math/tex; mode=display">\begin{equation}
Z_{on} = \sum_{(t,p) \in \text{Types} \times \text{Periods}}{\text{base_cost}_t*\text{ngen}_{t,p}}
\end{equation}</script><span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;msub&gt;&lt;mi&gt;Z&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;e&lt;/mi&gt;&lt;mi&gt;x&lt;/mi&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mi&gt;r&lt;/mi&gt;&lt;mi&gt;a&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Types&lt;/mtext&gt;&lt;mo&gt;&amp;#x00D7;&lt;/mo&gt;&lt;mtext&gt;Periods&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;per_mw_cost&lt;/mtext&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;output&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;min_load&lt;/mtext&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-62-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-502" style="width: 33.336em; display: inline-block;"><span style="display: inline-block; position: relative; width: 27.741em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.074em, 1027.62em, 3.812em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-503"><span class="msubsup" id="MathJax-Span-504"><span style="display: inline-block; position: relative; width: 2.443em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-505" style="font-family: MathJax_Math-italic;">Z<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.658em;"><span class="texatom" id="MathJax-Span-506"><span class="mrow" id="MathJax-Span-507"><span class="mi" id="MathJax-Span-508" style="font-size: 70.7%; font-family: MathJax_Math-italic;">e</span><span class="mi" id="MathJax-Span-509" style="font-size: 70.7%; font-family: MathJax_Math-italic;">x</span><span class="mi" id="MathJax-Span-510" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mi" id="MathJax-Span-511" style="font-size: 70.7%; font-family: MathJax_Math-italic;">r</span><span class="mi" id="MathJax-Span-512" style="font-size: 70.7%; font-family: MathJax_Math-italic;">a</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-513" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="munderover" id="MathJax-Span-514" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 6.551em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 2.562em;"><span class="mo" id="MathJax-Span-515" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.277em, 1006.55em, 4.467em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-516"><span class="mrow" id="MathJax-Span-517"><span class="mo" id="MathJax-Span-518" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-519" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-520" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-521" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-522" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-523" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-524" style="font-size: 70.7%; font-family: MathJax_Main;">Types</span><span class="mo" id="MathJax-Span-525" style="font-size: 70.7%; font-family: MathJax_Main;">×</span><span class="mtext" id="MathJax-Span-526" style="font-size: 70.7%; font-family: MathJax_Main;">Periods</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-527" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-528"><span class="msubsup" id="MathJax-Span-529"><span style="display: inline-block; position: relative; width: 6.015em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1005.6em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-530" style="font-family: MathJax_Main;">per_mw_cost</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 5.658em;"><span class="mi" id="MathJax-Span-531" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-532" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="mo" id="MathJax-Span-533" style="font-family: MathJax_Main; padding-left: 0.241em;">(</span><span class="msubsup" id="MathJax-Span-534"><span style="display: inline-block; position: relative; width: 3.812em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1002.92em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-535" style="font-family: MathJax_Main;">output</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.92em;"><span class="texatom" id="MathJax-Span-536"><span class="mrow" id="MathJax-Span-537"><span class="mi" id="MathJax-Span-538" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-539" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-540" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-541" style="font-family: MathJax_Main; padding-left: 0.241em;">−</span><span class="msubsup" id="MathJax-Span-542" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 4.348em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.99em, 4.229em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-543" style="font-family: MathJax_Main;">min_load</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 3.991em;"><span class="mi" id="MathJax-Span-544" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mo" id="MathJax-Span-545" style="font-family: MathJax_Main;">)</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.782em; border-left: 0px solid; width: 0px; height: 3.004em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>Z</mi><mrow class="MJX-TeXAtom-ORD"><mi>e</mi><mi>x</mi><mi>t</mi><mi>r</mi><mi>a</mi></mrow></msub><mo>=</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mo stretchy="false">(</mo><mi>t</mi><mo>,</mo><mi>p</mi><mo stretchy="false">)</mo><mo>∈</mo><mtext>Types</mtext><mo>×</mo><mtext>Periods</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>per_mw_cost</mtext><mi>t</mi></msub><mo>∗</mo><mo stretchy="false">(</mo><msub><mtext>output</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>p</mi></mrow></msub><mo>−</mo><msub><mtext>min_load</mtext><mi>t</mi></msub></mrow><mo stretchy="false">)</mo></math></span></span></div><script id="MathJax-Element-62" type="math/tex; mode=display">\begin{equation}
Z_{extra} = \sum_{(t,p) \in \text{Types} \times \text{Periods}}{\text{per_mw_cost}_t*(\text{output}_{t,p} - \text{min_load}_t})
\end{equation}</script><span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;msub&gt;&lt;mi&gt;Z&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mi&gt;a&lt;/mi&gt;&lt;mi&gt;r&lt;/mi&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mi&gt;u&lt;/mi&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Types&lt;/mtext&gt;&lt;mo&gt;&amp;#x00D7;&lt;/mo&gt;&lt;mtext&gt;Periods&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;startup_cost&lt;/mtext&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;nstart&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;/math&gt;' id="MathJax-Element-63-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-546" style="width: 25.658em; display: inline-block;"><span style="display: inline-block; position: relative; width: 21.372em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.074em, 1021.37em, 3.812em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-547"><span class="msubsup" id="MathJax-Span-548"><span style="display: inline-block; position: relative; width: 3.039em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-549" style="font-family: MathJax_Math-italic;">Z<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.658em;"><span class="texatom" id="MathJax-Span-550"><span class="mrow" id="MathJax-Span-551"><span class="mi" id="MathJax-Span-552" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mi" id="MathJax-Span-553" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mi" id="MathJax-Span-554" style="font-size: 70.7%; font-family: MathJax_Math-italic;">a</span><span class="mi" id="MathJax-Span-555" style="font-size: 70.7%; font-family: MathJax_Math-italic;">r</span><span class="mi" id="MathJax-Span-556" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mi" id="MathJax-Span-557" style="font-size: 70.7%; font-family: MathJax_Math-italic;">u</span><span class="mi" id="MathJax-Span-558" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-559" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="munderover" id="MathJax-Span-560" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 6.551em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 2.562em;"><span class="mo" id="MathJax-Span-561" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.277em, 1006.55em, 4.467em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-562"><span class="mrow" id="MathJax-Span-563"><span class="mo" id="MathJax-Span-564" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-565" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-566" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-567" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-568" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-569" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-570" style="font-size: 70.7%; font-family: MathJax_Main;">Types</span><span class="mo" id="MathJax-Span-571" style="font-size: 70.7%; font-family: MathJax_Main;">×</span><span class="mtext" id="MathJax-Span-572" style="font-size: 70.7%; font-family: MathJax_Main;">Periods</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-573" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-574"><span class="msubsup" id="MathJax-Span-575"><span style="display: inline-block; position: relative; width: 5.717em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1005.36em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-576" style="font-family: MathJax_Main;">startup_cost</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 5.42em;"><span class="mi" id="MathJax-Span-577" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-578" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="msubsup" id="MathJax-Span-579" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 3.515em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1002.56em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-580" style="font-family: MathJax_Main;">nstart</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 2.622em;"><span class="texatom" id="MathJax-Span-581"><span class="mrow" id="MathJax-Span-582"><span class="mi" id="MathJax-Span-583" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-584" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-585" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.782em; border-left: 0px solid; width: 0px; height: 3.004em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>Z</mi><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mi>t</mi><mi>a</mi><mi>r</mi><mi>t</mi><mi>u</mi><mi>p</mi></mrow></msub><mo>=</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mo stretchy="false">(</mo><mi>t</mi><mo>,</mo><mi>p</mi><mo stretchy="false">)</mo><mo>∈</mo><mtext>Types</mtext><mo>×</mo><mtext>Periods</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>startup_cost</mtext><mi>t</mi></msub><mo>∗</mo><msub><mtext>nstart</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>p</mi></mrow></msub></mrow></math></span></span></div><script id="MathJax-Element-63" type="math/tex; mode=display">\begin{equation}
Z_{startup} = \sum_{(t,p) \in \text{Types} \times \text{Periods}}{\text{startup_cost}_t*\text{nstart}_{t,p}}
\end{equation}</script><span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;msub&gt;&lt;mi&gt;Z&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;mi&gt;r&lt;/mi&gt;&lt;mi&gt;o&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;HydroUnits&lt;/mtext&gt;&lt;mo&gt;&amp;#x00D7;&lt;/mo&gt;&lt;mtext&gt;Periods&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;hydro_cost&lt;/mtext&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;hydro&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;/math&gt;' id="MathJax-Element-64-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-586" style="width: 26.908em; display: inline-block;"><span style="display: inline-block; position: relative; width: 22.384em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.074em, 1022.38em, 3.812em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-587"><span class="msubsup" id="MathJax-Span-588"><span style="display: inline-block; position: relative; width: 2.562em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-589" style="font-family: MathJax_Math-italic;">Z<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.658em;"><span class="texatom" id="MathJax-Span-590"><span class="mrow" id="MathJax-Span-591"><span class="mi" id="MathJax-Span-592" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span><span class="mi" id="MathJax-Span-593" style="font-size: 70.7%; font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mi" id="MathJax-Span-594" style="font-size: 70.7%; font-family: MathJax_Math-italic;">d<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mi" id="MathJax-Span-595" style="font-size: 70.7%; font-family: MathJax_Math-italic;">r</span><span class="mi" id="MathJax-Span-596" style="font-size: 70.7%; font-family: MathJax_Math-italic;">o</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-597" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="munderover" id="MathJax-Span-598" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 8.455em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 3.515em;"><span class="mo" id="MathJax-Span-599" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.277em, 1008.4em, 4.467em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-600"><span class="mrow" id="MathJax-Span-601"><span class="mo" id="MathJax-Span-602" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-603" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span><span class="mo" id="MathJax-Span-604" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-605" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-606" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-607" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-608" style="font-size: 70.7%; font-family: MathJax_Main;">HydroUnits</span><span class="mo" id="MathJax-Span-609" style="font-size: 70.7%; font-family: MathJax_Main;">×</span><span class="mtext" id="MathJax-Span-610" style="font-size: 70.7%; font-family: MathJax_Main;">Periods</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-611" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-612"><span class="msubsup" id="MathJax-Span-613"><span style="display: inline-block; position: relative; width: 5.241em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1004.71em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-614" style="font-family: MathJax_Main;">hydro_cost</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 4.765em;"><span class="mi" id="MathJax-Span-615" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-616" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="msubsup" id="MathJax-Span-617" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 3.574em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1002.5em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-618" style="font-family: MathJax_Main;">hydro</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.562em;"><span class="texatom" id="MathJax-Span-619"><span class="mrow" id="MathJax-Span-620"><span class="mi" id="MathJax-Span-621" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span><span class="mo" id="MathJax-Span-622" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-623" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.782em; border-left: 0px solid; width: 0px; height: 3.004em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>Z</mi><mrow class="MJX-TeXAtom-ORD"><mi>h</mi><mi>y</mi><mi>d</mi><mi>r</mi><mi>o</mi></mrow></msub><mo>=</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mo stretchy="false">(</mo><mi>h</mi><mo>,</mo><mi>p</mi><mo stretchy="false">)</mo><mo>∈</mo><mtext>HydroUnits</mtext><mo>×</mo><mtext>Periods</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>hydro_cost</mtext><mi>h</mi></msub><mo>∗</mo><msub><mtext>hydro</mtext><mrow class="MJX-TeXAtom-ORD"><mi>h</mi><mo>,</mo><mi>p</mi></mrow></msub></mrow></math></span></span></div><script id="MathJax-Element-64" type="math/tex; mode=display">\begin{equation}
Z_{hydro} = \sum_{(h,p) \in \text{HydroUnits} \times \text{Periods}}{\text{hydro_cost}_h*\text{hydro}_{h,p}}
\end{equation}</script><span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;msub&gt;&lt;mi&gt;Z&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;mi&gt;y&lt;/mi&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;mi&gt;r&lt;/mi&gt;&lt;mi&gt;o&lt;/mi&gt;&lt;mi mathvariant="normal"&gt;&amp;#x005F;&lt;/mi&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mi&gt;a&lt;/mi&gt;&lt;mi&gt;r&lt;/mi&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mi&gt;u&lt;/mi&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;HydroUnits&lt;/mtext&gt;&lt;mo&gt;&amp;#x00D7;&lt;/mo&gt;&lt;mtext&gt;Periods&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;hydro_startup_cost&lt;/mtext&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;hydro_start&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;/math&gt;' id="MathJax-Element-65-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-624" style="width: 37.503em; display: inline-block;"><span style="display: inline-block; position: relative; width: 31.253em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.074em, 1031.25em, 3.812em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-625"><span class="msubsup" id="MathJax-Span-626"><span style="display: inline-block; position: relative; width: 5.182em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-627" style="font-family: MathJax_Math-italic;">Z<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.658em;"><span class="texatom" id="MathJax-Span-628"><span class="mrow" id="MathJax-Span-629"><span class="mi" id="MathJax-Span-630" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span><span class="mi" id="MathJax-Span-631" style="font-size: 70.7%; font-family: MathJax_Math-italic;">y<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mi" id="MathJax-Span-632" style="font-size: 70.7%; font-family: MathJax_Math-italic;">d<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mi" id="MathJax-Span-633" style="font-size: 70.7%; font-family: MathJax_Math-italic;">r</span><span class="mi" id="MathJax-Span-634" style="font-size: 70.7%; font-family: MathJax_Math-italic;">o</span><span class="mi" id="MathJax-Span-635" style="font-size: 70.7%; font-family: MathJax_Main;">_</span><span class="mi" id="MathJax-Span-636" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mi" id="MathJax-Span-637" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mi" id="MathJax-Span-638" style="font-size: 70.7%; font-family: MathJax_Math-italic;">a</span><span class="mi" id="MathJax-Span-639" style="font-size: 70.7%; font-family: MathJax_Math-italic;">r</span><span class="mi" id="MathJax-Span-640" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mi" id="MathJax-Span-641" style="font-size: 70.7%; font-family: MathJax_Math-italic;">u</span><span class="mi" id="MathJax-Span-642" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-643" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="munderover" id="MathJax-Span-644" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 8.455em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 3.515em;"><span class="mo" id="MathJax-Span-645" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.277em, 1008.4em, 4.467em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-646"><span class="mrow" id="MathJax-Span-647"><span class="mo" id="MathJax-Span-648" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-649" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span><span class="mo" id="MathJax-Span-650" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-651" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-652" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-653" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-654" style="font-size: 70.7%; font-family: MathJax_Main;">HydroUnits</span><span class="mo" id="MathJax-Span-655" style="font-size: 70.7%; font-family: MathJax_Main;">×</span><span class="mtext" id="MathJax-Span-656" style="font-size: 70.7%; font-family: MathJax_Main;">Periods</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-657" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-658"><span class="msubsup" id="MathJax-Span-659"><span style="display: inline-block; position: relative; width: 8.932em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1008.4em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-660" style="font-family: MathJax_Main;">hydro_startup_cost</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 8.455em;"><span class="mi" id="MathJax-Span-661" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-662" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="msubsup" id="MathJax-Span-663" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 6.134em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1005.06em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-664" style="font-family: MathJax_Main;">hydro_start</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 5.122em;"><span class="texatom" id="MathJax-Span-665"><span class="mrow" id="MathJax-Span-666"><span class="mi" id="MathJax-Span-667" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span><span class="mo" id="MathJax-Span-668" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-669" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.782em; border-left: 0px solid; width: 0px; height: 3.004em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>Z</mi><mrow class="MJX-TeXAtom-ORD"><mi>h</mi><mi>y</mi><mi>d</mi><mi>r</mi><mi>o</mi><mi mathvariant="normal">_</mi><mi>s</mi><mi>t</mi><mi>a</mi><mi>r</mi><mi>t</mi><mi>u</mi><mi>p</mi></mrow></msub><mo>=</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mo stretchy="false">(</mo><mi>h</mi><mo>,</mo><mi>p</mi><mo stretchy="false">)</mo><mo>∈</mo><mtext>HydroUnits</mtext><mo>×</mo><mtext>Periods</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>hydro_startup_cost</mtext><mi>h</mi></msub><mo>∗</mo><msub><mtext>hydro_start</mtext><mrow class="MJX-TeXAtom-ORD"><mi>h</mi><mo>,</mo><mi>p</mi></mrow></msub></mrow></math></span></span></div><script id="MathJax-Element-65" type="math/tex; mode=display">\begin{equation}
Z_{hydro\_startup} = \sum_{(h,p) \in \text{HydroUnits} \times \text{Periods}}{\text{hydro_startup_cost}_h*\text{hydro_start}_{h,p}}
\end{equation}</script><h3 id="Constraints">Constraints<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/electrical_power_generation_1_2/electrical_power2.html#Constraints">¶</a></h3><ul>
<li><strong>Available generators</strong>: Number of generators used must be less than or equal to the number available.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;msub&gt;&lt;mtext&gt;ngen&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2264;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;generators&lt;/mtext&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Types&lt;/mtext&gt;&lt;mo&gt;&amp;#x00D7;&lt;/mo&gt;&lt;mtext&gt;Periods&lt;/mtext&gt;&lt;/math&gt;' id="MathJax-Element-66-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-670" style="width: 25.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 21.193em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1021.13em, 2.741em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-671"><span class="msubsup" id="MathJax-Span-672"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1002.03em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-673" style="font-family: MathJax_Main;">ngen</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.086em;"><span class="texatom" id="MathJax-Span-674"><span class="mrow" id="MathJax-Span-675"><span class="mi" id="MathJax-Span-676" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-677" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-678" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-679" style="font-family: MathJax_Main; padding-left: 0.301em;">≤</span><span class="msubsup" id="MathJax-Span-680" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 4.824em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1004.47em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-681" style="font-family: MathJax_Main;">generators</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 4.527em;"><span class="mi" id="MathJax-Span-682" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-683" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-684" style="font-family: MathJax_Main;">∀</span><span class="mo" id="MathJax-Span-685" style="font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-686" style="font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-687" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-688" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">p</span><span class="mo" id="MathJax-Span-689" style="font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-690" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-691" style="font-family: MathJax_Main; padding-left: 0.301em;">Types</span><span class="mo" id="MathJax-Span-692" style="font-family: MathJax_Main; padding-left: 0.241em;">×</span><span class="mtext" id="MathJax-Span-693" style="font-family: MathJax_Main; padding-left: 0.241em;">Periods</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.496em; border-left: 0px solid; width: 0px; height: 1.504em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>ngen</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>p</mi></mrow></msub><mo>≤</mo><msub><mtext>generators</mtext><mi>t</mi></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mo stretchy="false">(</mo><mi>t</mi><mo>,</mo><mi>p</mi><mo stretchy="false">)</mo><mo>∈</mo><mtext>Types</mtext><mo>×</mo><mtext>Periods</mtext></math></span></span></div><script id="MathJax-Element-66" type="math/tex; mode=display">\begin{equation}
\text{ngen}_{t,p} \leq \text{generators}_t  \quad \forall (t,p) \in \text{Types} \times \text{Periods}
\end{equation}</script><ul>
<li><strong>Demand</strong>: Total power generated across all generator types must meet anticipated demand plus pumping for each time period <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-67-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-694" style="width: 0.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.479em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.48em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-695"><span class="mi" id="MathJax-Span-696" style="font-family: MathJax_Math-italic;">p</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>p</mi></math></span></span><script id="MathJax-Element-67" type="math/tex">p</script>.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Types&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;output&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;HydroUnits&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;hydro_load&lt;/mtext&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;hydro&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mo&gt;&amp;#x2265;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;demand&lt;/mtext&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;pumping&lt;/mtext&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Periods&lt;/mtext&gt;&lt;/math&gt;' id="MathJax-Element-68-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-697" style="width: 47.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 39.824em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.074em, 1039.77em, 3.753em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-698"><span class="munderover" id="MathJax-Span-699"><span style="display: inline-block; position: relative; width: 2.622em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.598em;"><span class="mo" id="MathJax-Span-700" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1002.56em, 4.408em, -999.997em); top: -2.914em; left: 0em;"><span class="texatom" id="MathJax-Span-701"><span class="mrow" id="MathJax-Span-702"><span class="mi" id="MathJax-Span-703" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-704" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-705" style="font-size: 70.7%; font-family: MathJax_Main;">Types</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-706" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-707"><span class="msubsup" id="MathJax-Span-708"><span style="display: inline-block; position: relative; width: 3.812em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1002.92em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-709" style="font-family: MathJax_Main;">output</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.92em;"><span class="texatom" id="MathJax-Span-710"><span class="mrow" id="MathJax-Span-711"><span class="mi" id="MathJax-Span-712" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-713" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-714" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mo" id="MathJax-Span-715" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="munderover" id="MathJax-Span-716" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 4.467em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 1.551em;"><span class="mo" id="MathJax-Span-717" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1004.47em, 4.408em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-718"><span class="mrow" id="MathJax-Span-719"><span class="mi" id="MathJax-Span-720" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span><span class="mo" id="MathJax-Span-721" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-722" style="font-size: 70.7%; font-family: MathJax_Main;">HydroUnits</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-723" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-724"><span class="msubsup" id="MathJax-Span-725"><span style="display: inline-block; position: relative; width: 5.36em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1004.82em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-726" style="font-family: MathJax_Main;">hydro_load</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 4.884em;"><span class="mi" id="MathJax-Span-727" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-728" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="msubsup" id="MathJax-Span-729" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 3.574em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1002.5em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-730" style="font-family: MathJax_Main;">hydro</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.562em;"><span class="texatom" id="MathJax-Span-731"><span class="mrow" id="MathJax-Span-732"><span class="mi" id="MathJax-Span-733" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span><span class="mo" id="MathJax-Span-734" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-735" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mo" id="MathJax-Span-736" style="font-family: MathJax_Main; padding-left: 0.301em;">≥</span><span class="msubsup" id="MathJax-Span-737" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 3.872em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.46em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-738" style="font-family: MathJax_Main;">demand</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 3.455em;"><span class="mi" id="MathJax-Span-739" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-740" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="msubsup" id="MathJax-Span-741" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 4.289em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1003.81em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-742" style="font-family: MathJax_Main;">pumping</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 3.812em;"><span class="mi" id="MathJax-Span-743" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-744" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-745" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-746" style="font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-747" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-748" style="font-family: MathJax_Main; padding-left: 0.301em;">Periods</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.711em; border-left: 0px solid; width: 0px; height: 2.932em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>∈</mo><mtext>Types</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>output</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>p</mi></mrow></msub></mrow><mo>+</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>h</mi><mo>∈</mo><mtext>HydroUnits</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>hydro_load</mtext><mi>h</mi></msub><mo>∗</mo><msub><mtext>hydro</mtext><mrow class="MJX-TeXAtom-ORD"><mi>h</mi><mo>,</mo><mi>p</mi></mrow></msub></mrow><mo>≥</mo><msub><mtext>demand</mtext><mi>p</mi></msub><mo>+</mo><msub><mtext>pumping</mtext><mi>p</mi></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>p</mi><mo>∈</mo><mtext>Periods</mtext></math></span></span></div><script id="MathJax-Element-68" type="math/tex; mode=display">\begin{equation}
\sum_{t \in \text{Types}}{\text{output}_{t,p}} +
\sum_{h \in \text{HydroUnits}}{\text{hydro_load}_h*\text{hydro}_{h,p}} \geq
\text{demand}_p + \text{pumping}_p \quad \forall p \in \text{Periods}
\end{equation}</script><ul>
<li><strong>Min/max generation</strong>: Power generation must respect thermal generator min/max values.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;msub&gt;&lt;mtext&gt;output&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2265;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;min_output&lt;/mtext&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;ngen&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Types&lt;/mtext&gt;&lt;mo&gt;&amp;#x00D7;&lt;/mo&gt;&lt;mtext&gt;Periods&lt;/mtext&gt;&lt;/math&gt;' id="MathJax-Element-69-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-749" style="width: 31.967em; display: inline-block;"><span style="display: inline-block; position: relative; width: 26.61em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1026.55em, 2.741em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-750"><span class="msubsup" id="MathJax-Span-751"><span style="display: inline-block; position: relative; width: 3.812em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1002.92em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-752" style="font-family: MathJax_Main;">output</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.92em;"><span class="texatom" id="MathJax-Span-753"><span class="mrow" id="MathJax-Span-754"><span class="mi" id="MathJax-Span-755" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-756" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-757" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-758" style="font-family: MathJax_Main; padding-left: 0.301em;">≥</span><span class="msubsup" id="MathJax-Span-759" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 5.42em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1005.06em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-760" style="font-family: MathJax_Main;">min_output</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 5.122em;"><span class="mi" id="MathJax-Span-761" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-762" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="msubsup" id="MathJax-Span-763" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1002.03em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-764" style="font-family: MathJax_Main;">ngen</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.086em;"><span class="texatom" id="MathJax-Span-765"><span class="mrow" id="MathJax-Span-766"><span class="mi" id="MathJax-Span-767" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-768" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-769" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-770" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-771" style="font-family: MathJax_Main;">∀</span><span class="mo" id="MathJax-Span-772" style="font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-773" style="font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-774" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-775" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">p</span><span class="mo" id="MathJax-Span-776" style="font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-777" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-778" style="font-family: MathJax_Main; padding-left: 0.301em;">Types</span><span class="mo" id="MathJax-Span-779" style="font-family: MathJax_Main; padding-left: 0.241em;">×</span><span class="mtext" id="MathJax-Span-780" style="font-family: MathJax_Main; padding-left: 0.241em;">Periods</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.496em; border-left: 0px solid; width: 0px; height: 1.504em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>output</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>p</mi></mrow></msub><mo>≥</mo><msub><mtext>min_output</mtext><mi>t</mi></msub><mo>∗</mo><msub><mtext>ngen</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>p</mi></mrow></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mo stretchy="false">(</mo><mi>t</mi><mo>,</mo><mi>p</mi><mo stretchy="false">)</mo><mo>∈</mo><mtext>Types</mtext><mo>×</mo><mtext>Periods</mtext></math></span></span></div><script id="MathJax-Element-69" type="math/tex; mode=display">\begin{equation}
\text{output}_{t,p} \geq \text{min_output}_t*\text{ngen}_{t,p} \quad \forall (t,p) \in \text{Types} \times \text{Periods}
\end{equation}</script><span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;msub&gt;&lt;mtext&gt;output&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2264;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;max_output&lt;/mtext&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;ngen&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Types&lt;/mtext&gt;&lt;mo&gt;&amp;#x00D7;&lt;/mo&gt;&lt;mtext&gt;Periods&lt;/mtext&gt;&lt;/math&gt;' id="MathJax-Element-70-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-781" style="width: 32.265em; display: inline-block;"><span style="display: inline-block; position: relative; width: 26.848em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1026.79em, 2.741em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-782"><span class="msubsup" id="MathJax-Span-783"><span style="display: inline-block; position: relative; width: 3.812em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1002.92em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-784" style="font-family: MathJax_Main;">output</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.92em;"><span class="texatom" id="MathJax-Span-785"><span class="mrow" id="MathJax-Span-786"><span class="mi" id="MathJax-Span-787" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-788" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-789" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-790" style="font-family: MathJax_Main; padding-left: 0.301em;">≤</span><span class="msubsup" id="MathJax-Span-791" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 5.658em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1005.24em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-792" style="font-family: MathJax_Main;">max_output</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 5.301em;"><span class="mi" id="MathJax-Span-793" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-794" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="msubsup" id="MathJax-Span-795" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1002.03em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-796" style="font-family: MathJax_Main;">ngen</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.086em;"><span class="texatom" id="MathJax-Span-797"><span class="mrow" id="MathJax-Span-798"><span class="mi" id="MathJax-Span-799" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-800" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-801" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-802" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-803" style="font-family: MathJax_Main;">∀</span><span class="mo" id="MathJax-Span-804" style="font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-805" style="font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-806" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-807" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">p</span><span class="mo" id="MathJax-Span-808" style="font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-809" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-810" style="font-family: MathJax_Main; padding-left: 0.301em;">Types</span><span class="mo" id="MathJax-Span-811" style="font-family: MathJax_Main; padding-left: 0.241em;">×</span><span class="mtext" id="MathJax-Span-812" style="font-family: MathJax_Main; padding-left: 0.241em;">Periods</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.496em; border-left: 0px solid; width: 0px; height: 1.504em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>output</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>p</mi></mrow></msub><mo>≤</mo><msub><mtext>max_output</mtext><mi>t</mi></msub><mo>∗</mo><msub><mtext>ngen</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>p</mi></mrow></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mo stretchy="false">(</mo><mi>t</mi><mo>,</mo><mi>p</mi><mo stretchy="false">)</mo><mo>∈</mo><mtext>Types</mtext><mo>×</mo><mtext>Periods</mtext></math></span></span></div><script id="MathJax-Element-70" type="math/tex; mode=display">\begin{equation}
\text{output}_{t,p} \leq \text{max_output}_t*\text{ngen}_{t,p} \quad \forall (t,p) \in \text{Types} \times \text{Periods}
\end{equation}</script><ul>
<li><strong>Reserve</strong>: Selected thermal generators plus hydro units must be able to satisfy demand that is as much as 15% above the prediction.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Types&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;max_output&lt;/mtext&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;ngen&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;HydroUnits&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;hydro_load&lt;/mtext&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mo&gt;&amp;#x2265;&lt;/mo&gt;&lt;mn&gt;1.15&lt;/mn&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;demand&lt;/mtext&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Periods&lt;/mtext&gt;&lt;/math&gt;' id="MathJax-Element-71-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-813" style="width: 45.896em; display: inline-block;"><span style="display: inline-block; position: relative; width: 38.217em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.074em, 1038.16em, 3.753em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-814"><span class="munderover" id="MathJax-Span-815"><span style="display: inline-block; position: relative; width: 2.622em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.598em;"><span class="mo" id="MathJax-Span-816" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1002.56em, 4.408em, -999.997em); top: -2.914em; left: 0em;"><span class="texatom" id="MathJax-Span-817"><span class="mrow" id="MathJax-Span-818"><span class="mi" id="MathJax-Span-819" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-820" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-821" style="font-size: 70.7%; font-family: MathJax_Main;">Types</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-822" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-823"><span class="msubsup" id="MathJax-Span-824"><span style="display: inline-block; position: relative; width: 5.658em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1005.24em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-825" style="font-family: MathJax_Main;">max_output</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 5.301em;"><span class="mi" id="MathJax-Span-826" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-827" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="msubsup" id="MathJax-Span-828" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1002.03em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-829" style="font-family: MathJax_Main;">ngen</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.086em;"><span class="texatom" id="MathJax-Span-830"><span class="mrow" id="MathJax-Span-831"><span class="mi" id="MathJax-Span-832" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-833" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-834" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mo" id="MathJax-Span-835" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="munderover" id="MathJax-Span-836" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 4.467em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 1.551em;"><span class="mo" id="MathJax-Span-837" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1004.47em, 4.408em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-838"><span class="mrow" id="MathJax-Span-839"><span class="mi" id="MathJax-Span-840" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span><span class="mo" id="MathJax-Span-841" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-842" style="font-size: 70.7%; font-family: MathJax_Main;">HydroUnits</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-843" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-844"><span class="msubsup" id="MathJax-Span-845"><span style="display: inline-block; position: relative; width: 5.36em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1004.82em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-846" style="font-family: MathJax_Main;">hydro_load</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 4.884em;"><span class="mi" id="MathJax-Span-847" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mo" id="MathJax-Span-848" style="font-family: MathJax_Main; padding-left: 0.301em;">≥</span><span class="mn" id="MathJax-Span-849" style="font-family: MathJax_Main; padding-left: 0.301em;">1.15</span><span class="mo" id="MathJax-Span-850" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="msubsup" id="MathJax-Span-851" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 3.872em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.46em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-852" style="font-family: MathJax_Main;">demand</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 3.455em;"><span class="mi" id="MathJax-Span-853" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-854" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-855" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-856" style="font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-857" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-858" style="font-family: MathJax_Main; padding-left: 0.301em;">Periods</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.711em; border-left: 0px solid; width: 0px; height: 2.932em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>∈</mo><mtext>Types</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>max_output</mtext><mi>t</mi></msub><mo>∗</mo><msub><mtext>ngen</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>p</mi></mrow></msub></mrow><mo>+</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>h</mi><mo>∈</mo><mtext>HydroUnits</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>hydro_load</mtext><mi>h</mi></msub></mrow><mo>≥</mo><mn>1.15</mn><mo>∗</mo><msub><mtext>demand</mtext><mi>p</mi></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>p</mi><mo>∈</mo><mtext>Periods</mtext></math></span></span></div><script id="MathJax-Element-71" type="math/tex; mode=display">\begin{equation}
\sum_{t \in \text{Types}}{\text{max_output}_t*\text{ngen}_{t,p}} +
\sum_{h \in \text{HydroUnits}}{\text{hydro_load}_h} \geq 1.15 * \text{demand}_p \quad \forall p \in \text{Periods}
\end{equation}</script><ul>
<li><strong>Thermal startup</strong>: Establish relationship between number of active thermal generators and number of startups (use <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mi&gt;a&lt;/mi&gt;&lt;mi&gt;r&lt;/mi&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-72-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-859" style="width: 3.217em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.62em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-860"><span class="mi" id="MathJax-Span-861" style="font-family: MathJax_Math-italic;">s</span><span class="mi" id="MathJax-Span-862" style="font-family: MathJax_Math-italic;">t</span><span class="mi" id="MathJax-Span-863" style="font-family: MathJax_Math-italic;">a</span><span class="mi" id="MathJax-Span-864" style="font-family: MathJax_Math-italic;">r</span><span class="mi" id="MathJax-Span-865" style="font-family: MathJax_Math-italic;">t</span><span class="mn" id="MathJax-Span-866" style="font-family: MathJax_Main;">0</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>s</mi><mi>t</mi><mi>a</mi><mi>r</mi><mi>t</mi><mn>0</mn></math></span></span><script id="MathJax-Element-72" type="math/tex">start0</script> for period before the time horizon starts)</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;msub&gt;&lt;mtext&gt;ngen&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2264;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;ngen&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;nstart&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Types&lt;/mtext&gt;&lt;mo&gt;&amp;#x00D7;&lt;/mo&gt;&lt;mtext&gt;Periods&lt;/mtext&gt;&lt;/math&gt;' id="MathJax-Element-73-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-867" style="width: 30.122em; display: inline-block;"><span style="display: inline-block; position: relative; width: 25.063em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1025em, 2.741em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-868"><span class="msubsup" id="MathJax-Span-869"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1002.03em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-870" style="font-family: MathJax_Main;">ngen</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.086em;"><span class="texatom" id="MathJax-Span-871"><span class="mrow" id="MathJax-Span-872"><span class="mi" id="MathJax-Span-873" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-874" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-875" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-876" style="font-family: MathJax_Main; padding-left: 0.301em;">≤</span><span class="msubsup" id="MathJax-Span-877" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 3.872em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1002.03em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-878" style="font-family: MathJax_Main;">ngen</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.086em;"><span class="texatom" id="MathJax-Span-879"><span class="mrow" id="MathJax-Span-880"><span class="mi" id="MathJax-Span-881" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-882" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-883" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-884" style="font-size: 70.7%; font-family: MathJax_Main;">−</span><span class="mn" id="MathJax-Span-885" style="font-size: 70.7%; font-family: MathJax_Main;">1</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-886" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="msubsup" id="MathJax-Span-887" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 3.515em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1002.56em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-888" style="font-family: MathJax_Main;">nstart</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 2.622em;"><span class="texatom" id="MathJax-Span-889"><span class="mrow" id="MathJax-Span-890"><span class="mi" id="MathJax-Span-891" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-892" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-893" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-894" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-895" style="font-family: MathJax_Main;">∀</span><span class="mo" id="MathJax-Span-896" style="font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-897" style="font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-898" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-899" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">p</span><span class="mo" id="MathJax-Span-900" style="font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-901" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-902" style="font-family: MathJax_Main; padding-left: 0.301em;">Types</span><span class="mo" id="MathJax-Span-903" style="font-family: MathJax_Main; padding-left: 0.241em;">×</span><span class="mtext" id="MathJax-Span-904" style="font-family: MathJax_Main; padding-left: 0.241em;">Periods</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.496em; border-left: 0px solid; width: 0px; height: 1.504em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>ngen</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>p</mi></mrow></msub><mo>≤</mo><msub><mtext>ngen</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>p</mi><mo>−</mo><mn>1</mn></mrow></msub><mo>+</mo><msub><mtext>nstart</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>p</mi></mrow></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mo stretchy="false">(</mo><mi>t</mi><mo>,</mo><mi>p</mi><mo stretchy="false">)</mo><mo>∈</mo><mtext>Types</mtext><mo>×</mo><mtext>Periods</mtext></math></span></span></div><script id="MathJax-Element-73" type="math/tex; mode=display">\begin{equation}
\text{ngen}_{t,p} \leq \text{ngen}_{t,p-1} + \text{nstart}_{t,p} \quad \forall (t,p) \in \text{Types} \times \text{Periods}
\end{equation}</script><ul>
<li><strong>Hydro startup</strong>: Establish relationship between hydro generator state and number of hydro startups (assume hydro plants are off at the beginning of the horizon)</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;msub&gt;&lt;mtext&gt;hydro&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2264;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;hydro&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;hydro_start&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;HydroUnits&lt;/mtext&gt;&lt;mo&gt;&amp;#x00D7;&lt;/mo&gt;&lt;mtext&gt;Periods&lt;/mtext&gt;&lt;/math&gt;' id="MathJax-Element-74-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-905" style="width: 37.86em; display: inline-block;"><span style="display: inline-block; position: relative; width: 31.551em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1031.49em, 2.741em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-906"><span class="msubsup" id="MathJax-Span-907"><span style="display: inline-block; position: relative; width: 3.574em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1002.5em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-908" style="font-family: MathJax_Main;">hydro</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.562em;"><span class="texatom" id="MathJax-Span-909"><span class="mrow" id="MathJax-Span-910"><span class="mi" id="MathJax-Span-911" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span><span class="mo" id="MathJax-Span-912" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-913" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-914" style="font-family: MathJax_Main; padding-left: 0.301em;">≤</span><span class="msubsup" id="MathJax-Span-915" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 4.467em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1002.5em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-916" style="font-family: MathJax_Main;">hydro</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.562em;"><span class="texatom" id="MathJax-Span-917"><span class="mrow" id="MathJax-Span-918"><span class="mi" id="MathJax-Span-919" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span><span class="mo" id="MathJax-Span-920" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-921" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-922" style="font-size: 70.7%; font-family: MathJax_Main;">−</span><span class="mn" id="MathJax-Span-923" style="font-size: 70.7%; font-family: MathJax_Main;">1</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-924" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="msubsup" id="MathJax-Span-925" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 6.134em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1005.06em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-926" style="font-family: MathJax_Main;">hydro_start</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 5.122em;"><span class="texatom" id="MathJax-Span-927"><span class="mrow" id="MathJax-Span-928"><span class="mi" id="MathJax-Span-929" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span><span class="mo" id="MathJax-Span-930" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-931" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-932" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-933" style="font-family: MathJax_Main;">∀</span><span class="mo" id="MathJax-Span-934" style="font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-935" style="font-family: MathJax_Math-italic;">h</span><span class="mo" id="MathJax-Span-936" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-937" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">p</span><span class="mo" id="MathJax-Span-938" style="font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-939" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-940" style="font-family: MathJax_Main; padding-left: 0.301em;">HydroUnits</span><span class="mo" id="MathJax-Span-941" style="font-family: MathJax_Main; padding-left: 0.241em;">×</span><span class="mtext" id="MathJax-Span-942" style="font-family: MathJax_Main; padding-left: 0.241em;">Periods</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.496em; border-left: 0px solid; width: 0px; height: 1.504em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>hydro</mtext><mrow class="MJX-TeXAtom-ORD"><mi>h</mi><mo>,</mo><mi>p</mi></mrow></msub><mo>≤</mo><msub><mtext>hydro</mtext><mrow class="MJX-TeXAtom-ORD"><mi>h</mi><mo>,</mo><mi>p</mi><mo>−</mo><mn>1</mn></mrow></msub><mo>+</mo><msub><mtext>hydro_start</mtext><mrow class="MJX-TeXAtom-ORD"><mi>h</mi><mo>,</mo><mi>p</mi></mrow></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mo stretchy="false">(</mo><mi>h</mi><mo>,</mo><mi>p</mi><mo stretchy="false">)</mo><mo>∈</mo><mtext>HydroUnits</mtext><mo>×</mo><mtext>Periods</mtext></math></span></span></div><script id="MathJax-Element-74" type="math/tex; mode=display">\begin{equation}
\text{hydro}_{h,p} \leq \text{hydro}_{h,p-1} + \text{hydro_start}_{h,p} \quad \forall (h,p) \in \text{HydroUnits} \times \text{Periods}
\end{equation}</script><ul>
<li><p><strong>Reservoir height</strong>: Track reservoir height.   Note that the height at the end of the final time period must equal the height at the beginning of the first.</p>
</li>
<li><p>Reservoir level constraints: Height increases due to pumping activity and decreases due to hydroelectric generation.</p>
</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;msub&gt;&lt;mtext&gt;height&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;height&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;period_hours&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;pumping&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mo&gt;/&lt;/mo&gt;&lt;/mrow&gt;&lt;mn&gt;3000&lt;/mn&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;HydroUnits&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;period_hours&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;hydro_height_reduction&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;hydro&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Periods&lt;/mtext&gt;&lt;/math&gt;' id="MathJax-Element-75-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-943" style="width: 70.36em; display: inline-block;"><span style="display: inline-block; position: relative; width: 58.634em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.074em, 1058.57em, 3.753em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-944"><span class="msubsup" id="MathJax-Span-945"><span style="display: inline-block; position: relative; width: 3.158em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1002.68em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-946" style="font-family: MathJax_Main;">height</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.741em;"><span class="texatom" id="MathJax-Span-947"><span class="mrow" id="MathJax-Span-948"><span class="mi" id="MathJax-Span-949" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-950" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="msubsup" id="MathJax-Span-951" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 4.051em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1002.68em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-952" style="font-family: MathJax_Main;">height</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.741em;"><span class="texatom" id="MathJax-Span-953"><span class="mrow" id="MathJax-Span-954"><span class="mi" id="MathJax-Span-955" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-956" style="font-size: 70.7%; font-family: MathJax_Main;">−</span><span class="mn" id="MathJax-Span-957" style="font-size: 70.7%; font-family: MathJax_Main;">1</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-958" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="msubsup" id="MathJax-Span-959" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 6.074em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1005.6em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-960" style="font-family: MathJax_Main;">period_hours</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 5.598em;"><span class="texatom" id="MathJax-Span-961"><span class="mrow" id="MathJax-Span-962"><span class="mi" id="MathJax-Span-963" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-964" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="msubsup" id="MathJax-Span-965" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 4.289em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1003.81em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-966" style="font-family: MathJax_Main;">pumping</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 3.812em;"><span class="texatom" id="MathJax-Span-967"><span class="mrow" id="MathJax-Span-968"><span class="mi" id="MathJax-Span-969" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-970"><span class="mrow" id="MathJax-Span-971"><span class="mo" id="MathJax-Span-972" style="font-family: MathJax_Main;">/</span></span></span><span class="mn" id="MathJax-Span-973" style="font-family: MathJax_Main;">3000</span><span class="mo" id="MathJax-Span-974" style="font-family: MathJax_Main; padding-left: 0.241em;">−</span><span class="munderover" id="MathJax-Span-975" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 4.467em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 1.551em;"><span class="mo" id="MathJax-Span-976" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1004.47em, 4.408em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-977"><span class="mrow" id="MathJax-Span-978"><span class="mi" id="MathJax-Span-979" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span><span class="mo" id="MathJax-Span-980" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-981" style="font-size: 70.7%; font-family: MathJax_Main;">HydroUnits</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-982" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-983"><span class="msubsup" id="MathJax-Span-984"><span style="display: inline-block; position: relative; width: 6.074em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1005.6em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-985" style="font-family: MathJax_Main;">period_hours</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 5.598em;"><span class="texatom" id="MathJax-Span-986"><span class="mrow" id="MathJax-Span-987"><span class="mi" id="MathJax-Span-988" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-989" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="msubsup" id="MathJax-Span-990" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 10.836em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1010.36em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-991" style="font-family: MathJax_Main;">hydro_height_reduction</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 10.36em;"><span class="texatom" id="MathJax-Span-992"><span class="mrow" id="MathJax-Span-993"><span class="mi" id="MathJax-Span-994" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-995" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="msubsup" id="MathJax-Span-996" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 3.574em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1002.5em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-997" style="font-family: MathJax_Main;">hydro</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.562em;"><span class="texatom" id="MathJax-Span-998"><span class="mrow" id="MathJax-Span-999"><span class="mi" id="MathJax-Span-1000" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span><span class="mo" id="MathJax-Span-1001" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1002" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mspace" id="MathJax-Span-1003" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-1004" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-1005" style="font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-1006" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-1007" style="font-family: MathJax_Main; padding-left: 0.301em;">Periods</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.711em; border-left: 0px solid; width: 0px; height: 2.932em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>height</mtext><mrow class="MJX-TeXAtom-ORD"><mi>p</mi></mrow></msub><mo>=</mo><msub><mtext>height</mtext><mrow class="MJX-TeXAtom-ORD"><mi>p</mi><mo>−</mo><mn>1</mn></mrow></msub><mo>+</mo><msub><mtext>period_hours</mtext><mrow class="MJX-TeXAtom-ORD"><mi>p</mi></mrow></msub><mo>∗</mo><msub><mtext>pumping</mtext><mrow class="MJX-TeXAtom-ORD"><mi>p</mi></mrow></msub><mrow class="MJX-TeXAtom-ORD"><mo>/</mo></mrow><mn>3000</mn><mo>−</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>h</mi><mo>∈</mo><mtext>HydroUnits</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>period_hours</mtext><mrow class="MJX-TeXAtom-ORD"><mi>p</mi></mrow></msub><mo>∗</mo><msub><mtext>hydro_height_reduction</mtext><mrow class="MJX-TeXAtom-ORD"><mi>h</mi></mrow></msub><mo>∗</mo><msub><mtext>hydro</mtext><mrow class="MJX-TeXAtom-ORD"><mi>h</mi><mo>,</mo><mi>p</mi></mrow></msub></mrow><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>p</mi><mo>∈</mo><mtext>Periods</mtext></math></span></span></div><script id="MathJax-Element-75" type="math/tex; mode=display">\begin{equation}
\text{height}_{p} = \text{height}_{p-1}  + \text{period_hours}_{p}*\text{pumping}_{p}/3000 -
\sum_{h \in \text{HydroUnits}}{\text{period_hours}_{p}*\text{hydro_height_reduction}_{h}*\text{hydro}_{h,p}} \quad \forall p \in \text{Periods}
\end{equation}</script><ul>
<li>Cyclic constraint: Height at the first period must be equal to height at the last period.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display"><span class="MathJax MathJax_FullWidth" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;msub&gt;&lt;mtext&gt;height&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mi&gt;f&lt;/mi&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mi&gt;r&lt;/mi&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;height&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mi&gt;l&lt;/mi&gt;&lt;mi&gt;a&lt;/mi&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;period_hours&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mi&gt;f&lt;/mi&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mi&gt;r&lt;/mi&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;pumping&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mi&gt;f&lt;/mi&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mi&gt;r&lt;/mi&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mo&gt;/&lt;/mo&gt;&lt;/mrow&gt;&lt;mn&gt;3000&lt;/mn&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;HydroUnits&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;period_hours&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mi&gt;f&lt;/mi&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mi&gt;r&lt;/mi&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;hydro_height_reduction&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;hydro&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;h&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mi&gt;f&lt;/mi&gt;&lt;mi&gt;i&lt;/mi&gt;&lt;mi&gt;r&lt;/mi&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;/math&gt;' id="MathJax-Element-76-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1008" style="width: 100%; display: inline-block; min-width: 36.253em;"><span style="display: inline-block; position: relative; width: 100%; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(3.932em, 1030.18em, 8.158em, -999.997em); top: -4.878em; left: 0em; width: 100%;"><span class="mrow" id="MathJax-Span-1009"><span style="display: inline-block; position: relative; width: 100%; height: 0px;"><span style="position: absolute; clip: rect(3.039em, 1029.47em, 4.527em, -999.997em); top: -3.985em; left: 50%; margin-left: -14.759em;"><span class="msubsup" id="MathJax-Span-1010"><span style="display: inline-block; position: relative; width: 4.705em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1002.68em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1011" style="font-family: MathJax_Main;">height</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.741em;"><span class="texatom" id="MathJax-Span-1012"><span class="mrow" id="MathJax-Span-1013"><span class="mi" id="MathJax-Span-1014" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mi" id="MathJax-Span-1015" style="font-size: 70.7%; font-family: MathJax_Math-italic;">f<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mi" id="MathJax-Span-1016" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mi" id="MathJax-Span-1017" style="font-size: 70.7%; font-family: MathJax_Math-italic;">r</span><span class="mi" id="MathJax-Span-1018" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mi" id="MathJax-Span-1019" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1020" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="msubsup" id="MathJax-Span-1021" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 4.348em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1002.68em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1022" style="font-family: MathJax_Main;">height</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.741em;"><span class="texatom" id="MathJax-Span-1023"><span class="mrow" id="MathJax-Span-1024"><span class="mi" id="MathJax-Span-1025" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mi" id="MathJax-Span-1026" style="font-size: 70.7%; font-family: MathJax_Math-italic;">l</span><span class="mi" id="MathJax-Span-1027" style="font-size: 70.7%; font-family: MathJax_Math-italic;">a</span><span class="mi" id="MathJax-Span-1028" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mi" id="MathJax-Span-1029" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1030" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="msubsup" id="MathJax-Span-1031" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 7.622em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1005.6em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1032" style="font-family: MathJax_Main;">period_hours</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 5.598em;"><span class="texatom" id="MathJax-Span-1033"><span class="mrow" id="MathJax-Span-1034"><span class="mi" id="MathJax-Span-1035" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mi" id="MathJax-Span-1036" style="font-size: 70.7%; font-family: MathJax_Math-italic;">f<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mi" id="MathJax-Span-1037" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mi" id="MathJax-Span-1038" style="font-size: 70.7%; font-family: MathJax_Math-italic;">r</span><span class="mi" id="MathJax-Span-1039" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mi" id="MathJax-Span-1040" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1041" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="msubsup" id="MathJax-Span-1042" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 5.836em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1003.81em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1043" style="font-family: MathJax_Main;">pumping</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 3.812em;"><span class="texatom" id="MathJax-Span-1044"><span class="mrow" id="MathJax-Span-1045"><span class="mi" id="MathJax-Span-1046" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mi" id="MathJax-Span-1047" style="font-size: 70.7%; font-family: MathJax_Math-italic;">f<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mi" id="MathJax-Span-1048" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mi" id="MathJax-Span-1049" style="font-size: 70.7%; font-family: MathJax_Math-italic;">r</span><span class="mi" id="MathJax-Span-1050" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mi" id="MathJax-Span-1051" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-1052"><span class="mrow" id="MathJax-Span-1053"><span class="mo" id="MathJax-Span-1054" style="font-family: MathJax_Main;">/</span></span></span><span class="mn" id="MathJax-Span-1055" style="font-family: MathJax_Main;">3000</span><span class="mo" id="MathJax-Span-1056" style="font-family: MathJax_Main; padding-left: 0.241em;">−</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(2.86em, 1030.18em, 5.539em, -999.997em); top: -2.199em; left: 50%; margin-left: -15.057em;"><span class="munderover" id="MathJax-Span-1057" style=""><span style="display: inline-block; position: relative; width: 4.467em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 1.551em;"><span class="mo" id="MathJax-Span-1058" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1004.47em, 4.408em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-1059"><span class="mrow" id="MathJax-Span-1060"><span class="mi" id="MathJax-Span-1061" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span><span class="mo" id="MathJax-Span-1062" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-1063" style="font-size: 70.7%; font-family: MathJax_Main;">HydroUnits</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-1064" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-1065"><span class="msubsup" id="MathJax-Span-1066"><span style="display: inline-block; position: relative; width: 7.622em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1005.6em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1067" style="font-family: MathJax_Main;">period_hours</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 5.598em;"><span class="texatom" id="MathJax-Span-1068"><span class="mrow" id="MathJax-Span-1069"><span class="mi" id="MathJax-Span-1070" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mi" id="MathJax-Span-1071" style="font-size: 70.7%; font-family: MathJax_Math-italic;">f<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mi" id="MathJax-Span-1072" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mi" id="MathJax-Span-1073" style="font-size: 70.7%; font-family: MathJax_Math-italic;">r</span><span class="mi" id="MathJax-Span-1074" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mi" id="MathJax-Span-1075" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1076" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="msubsup" id="MathJax-Span-1077" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 10.836em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1010.36em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1078" style="font-family: MathJax_Main;">hydro_height_reduction</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 10.36em;"><span class="texatom" id="MathJax-Span-1079"><span class="mrow" id="MathJax-Span-1080"><span class="mi" id="MathJax-Span-1081" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-1082" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="msubsup" id="MathJax-Span-1083" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 5.122em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1002.5em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-1084" style="font-family: MathJax_Main;">hydro</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.562em;"><span class="texatom" id="MathJax-Span-1085"><span class="mrow" id="MathJax-Span-1086"><span class="mi" id="MathJax-Span-1087" style="font-size: 70.7%; font-family: MathJax_Math-italic;">h</span><span class="mo" id="MathJax-Span-1088" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-1089" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mi" id="MathJax-Span-1090" style="font-size: 70.7%; font-family: MathJax_Math-italic;">f<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mi" id="MathJax-Span-1091" style="font-size: 70.7%; font-family: MathJax_Math-italic;">i</span><span class="mi" id="MathJax-Span-1092" style="font-size: 70.7%; font-family: MathJax_Math-italic;">r</span><span class="mi" id="MathJax-Span-1093" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mi" id="MathJax-Span-1094" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 4.884em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -3.782em; border-left: 0px solid; width: 0px; height: 4.789em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>height</mtext><mrow class="MJX-TeXAtom-ORD"><mi>p</mi><mi>f</mi><mi>i</mi><mi>r</mi><mi>s</mi><mi>t</mi></mrow></msub><mo>=</mo><msub><mtext>height</mtext><mrow class="MJX-TeXAtom-ORD"><mi>p</mi><mi>l</mi><mi>a</mi><mi>s</mi><mi>t</mi></mrow></msub><mo>+</mo><msub><mtext>period_hours</mtext><mrow class="MJX-TeXAtom-ORD"><mi>p</mi><mi>f</mi><mi>i</mi><mi>r</mi><mi>s</mi><mi>t</mi></mrow></msub><mo>∗</mo><msub><mtext>pumping</mtext><mrow class="MJX-TeXAtom-ORD"><mi>p</mi><mi>f</mi><mi>i</mi><mi>r</mi><mi>s</mi><mi>t</mi></mrow></msub><mrow class="MJX-TeXAtom-ORD"><mo>/</mo></mrow><mn>3000</mn><mo>−</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>h</mi><mo>∈</mo><mtext>HydroUnits</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>period_hours</mtext><mrow class="MJX-TeXAtom-ORD"><mi>p</mi><mi>f</mi><mi>i</mi><mi>r</mi><mi>s</mi><mi>t</mi></mrow></msub><mo>∗</mo><msub><mtext>hydro_height_reduction</mtext><mrow class="MJX-TeXAtom-ORD"><mi>h</mi></mrow></msub><mo>∗</mo><msub><mtext>hydro</mtext><mrow class="MJX-TeXAtom-ORD"><mi>h</mi><mo>,</mo><mi>p</mi><mi>f</mi><mi>i</mi><mi>r</mi><mi>s</mi><mi>t</mi></mrow></msub></mrow></math></span></span></div><script id="MathJax-Element-76" type="math/tex; mode=display">\begin{equation}
\text{height}_{pfirst} = \text{height}_{plast}  + \text{period_hours}_{pfirst}*\text{pumping}_{pfirst}/3000 -
\sum_{h \in \text{HydroUnits}}{\text{period_hours}_{pfirst}*\text{hydro_height_reduction}_{h}*\text{hydro}_{h,pfirst}}
\end{equation}</script><hr/>
<h2 id="Python-Implementation">Python Implementation<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/electrical_power_generation_1_2/electrical_power2.html#Python-Implementation">¶</a></h2><p>We import the Gurobi Python Module and other Python libraries.</p>


In [ ]:

import numpy as np
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7.0 & Gurobi 9.0




<h2 id="Input-Data">Input Data<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/electrical_power_generation_1_2/electrical_power2.html#Input-Data">¶</a></h2><p>We define all the input data of the model.</p>


In [ ]:

# Parameters

ntypes = 3
nperiods = 5
maxstart0 = 5
hydrounits = 2

generators = [12, 10, 5]
period_hours = [6, 3, 6, 3, 6]
demand = [15000, 30000, 25000, 40000, 27000]
min_load = [850, 1250, 1500]
max_load = [2000, 1750, 4000]
base_cost = [1000, 2600, 3000]
per_mw_cost = [2, 1.3, 3]
startup_cost = [2000, 1000, 500]

hydro_load = [900, 1400]
hydro_cost = [90, 150]
hydro_height_reduction = [0.31, 0.47]
hydro_startup_cost = [1500, 1200]




<h2 id="Model-Deployment">Model Deployment<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/electrical_power_generation_1_2/electrical_power2.html#Model-Deployment">¶</a></h2><p>We create a model and the variables. For each time period, we have: an integer decision variable to capture the number of active generators of each type (ngen), an integer variable to capture the number of generators of each type we must start (nstart), a continuous decision variable to capture the total power output for each generator type (output), a binary decision variable that indicates whether a hydro unit is active (hydro), a binary decision variable that indicates whether a hydro unit must be started (hydrstart), a continuous decision variable that captures the amount of enery used to replenish the reservoir (pumping), and a continuous decision variable that captures the height of the reservoir (height).</p>


In [ ]:

model = gp.Model('PowerGeneration2')

ngen = model.addVars(ntypes, nperiods, vtype=GRB.INTEGER, name="ngen")
nstart = model.addVars(ntypes, nperiods, vtype=GRB.INTEGER, name="nstart")
output = model.addVars(ntypes, nperiods, vtype=GRB.CONTINUOUS, name="genoutput")

hydro = model.addVars(hydrounits, nperiods, vtype=GRB.BINARY, name="hydro")
hydrostart = model.addVars(hydrounits, nperiods, vtype=GRB.BINARY, name="hydrostart")
pumping = model.addVars(nperiods, vtype=GRB.CONTINUOUS, name="pumping")
height = model.addVars(nperiods, vtype=GRB.CONTINUOUS, name="height")




<p>Next we insert the constraints:</p>
<p>The number of active generators can't exceed the number of generators:</p>


In [ ]:

# Generator count

numgen = model.addConstrs(ngen[type, period] <= generators[type]
                         for type in range(ntypes) for period in range(nperiods))




<p>Total power output for a thermal generator type depends on the number of generators of that type that are active.</p>


In [ ]:

# Respect minimum and maximum output per generator type

min_output = model.addConstrs((output[type, period] >= min_load[type] * ngen[type, period])
                              for type in range(ntypes) for period in range(nperiods))

max_output = model.addConstrs((output[type, period] <= max_load[type] * ngen[type, period])
                              for type in range(ntypes) for period in range(nperiods))




<p>Total generator output (thermal plus hydro) for each time period must meet predicted demand plus pumping.</p>


In [ ]:

# Meet demand

meet_demand = model.addConstrs(gp.quicksum(output[type, period] for type in range(ntypes)) +
                               gp.quicksum(hydro_load[unit]*hydro[unit,period] for unit in range(hydrounits))
                               >= demand[period] + pumping[period]
                               for period in range(nperiods))




<p>Maintain appropriate reservoir levels</p>


In [ ]:

# Reservoir levels

reservoir = model.addConstrs(height[period] == height[period-1] + period_hours[period]*pumping[period]/3000 -
                             gp.quicksum(hydro_height_reduction[unit]*period_hours[period]*hydro[unit,period] for unit in range(hydrounits))
                             for period in range(1,nperiods))

# cyclic - height at end must equal height at beginning
reservoir0 = model.addConstr(height[0] == height[nperiods-1] + period_hours[0]*pumping[0]/3000 -
                             gp.quicksum(hydro_height_reduction[unit]*period_hours[0]*hydro[unit,0]
                             for unit in range(hydrounits)))




<p>Selected generators must be able to cope with an excess of demand.</p>


In [ ]:

# Provide sufficient reserve capacity

reserve = model.addConstrs(gp.quicksum(max_load[type]*ngen[type, period] for type in range(ntypes)) >=
                           1.15*demand[period] - sum(hydro_load)
                           for period in range(nperiods))




<p>Connect the decision variables that capture active thermal generators with the decision variables that count startups.</p>


In [ ]:

# Startup constraint

startup0 = model.addConstrs((ngen[type,0] <= maxstart0 + nstart[type,0])
                            for type in range(ntypes))

startup = model.addConstrs((ngen[type,period] <= ngen[type,period-1] + nstart[type,period])
                           for type in range(ntypes) for period in range(1,nperiods))




<p>Connect hydro decision variables with hydro startup decision variables.</p>


In [ ]:

# Hydro startup constraint

hydro_startup0 = model.addConstrs((hydro[unit,0] <= hydrostart[unit,0])
                                    for unit in range(hydrounits))

hydro_startup = model.addConstrs((hydro[unit,period] <= hydro[unit,period-1] + hydrostart[unit,period])
                           for unit in range(hydrounits) for period in range(1,nperiods))




<p>Objective: minimize total cost.  Cost consists of five components: the cost for running active thermal generation units, the cost to generate power beyond the minimum for each unit, the cost to start up thermal generation units, the cost to operate hydro units, and the cost to start up hydro units.</p>


In [ ]:

# Objective: minimize total cost

active = gp.quicksum(base_cost[type]*period_hours[period]*ngen[type,period]
                    for type in range(ntypes) for period in range(nperiods))

per_mw = gp.quicksum(per_mw_cost[type]*period_hours[period]*(output[type,period] - min_load[type]*ngen[type,period])
                       for type in range(ntypes) for period in range(nperiods))

startup_obj = gp.quicksum(startup_cost[type]*nstart[type,period]
                         for type in range(ntypes) for period in range(nperiods))

hydro_active = gp.quicksum(hydro_cost[unit]*period_hours[period]*hydro[unit,period]
                           for unit in range(hydrounits) for period in range(nperiods))

hydro_startup = gp.quicksum(hydro_startup_cost[unit]*hydrostart[unit,period]
                            for unit in range(hydrounits) for period in range(nperiods))

model.setObjective(active + per_mw + startup_obj + hydro_active + hydro_startup)




<p>Next, we start the optimization and Gurobi finds the optimal solution.</p>


In [ ]:

model.optimize()




<hr/>
<h2 id="Analysis">Analysis<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/electrical_power_generation_1_2/electrical_power2.html#Analysis">¶</a></h2><p>The anticipated demand for electricity over the 24-hour time window can be met for a total cost of <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;000&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;630&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-77-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1095" style="width: 5.955em; display: inline-block;"><span style="display: inline-block; position: relative; width: 4.943em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1004.88em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1096"><span class="mi" id="MathJax-Span-1097" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-1098" style="font-family: MathJax_Main;">1</span><span class="mo" id="MathJax-Span-1099" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-1100" style="font-family: MathJax_Main; padding-left: 0.182em;">000</span><span class="mo" id="MathJax-Span-1101" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-1102" style="font-family: MathJax_Main; padding-left: 0.182em;">630</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.289em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>1</mn><mo>,</mo><mn>000</mn><mo>,</mo><mn>630</mn></math></span></span><script id="MathJax-Element-77" type="math/tex">\$1,000,630</script>, as compared with the <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;002&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;540&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-78-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1103" style="width: 5.955em; display: inline-block;"><span style="display: inline-block; position: relative; width: 4.943em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1004.88em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-1104"><span class="mi" id="MathJax-Span-1105" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-1106" style="font-family: MathJax_Main;">1</span><span class="mo" id="MathJax-Span-1107" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-1108" style="font-family: MathJax_Main; padding-left: 0.182em;">002</span><span class="mo" id="MathJax-Span-1109" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-1110" style="font-family: MathJax_Main; padding-left: 0.182em;">540</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.289em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>1</mn><mo>,</mo><mn>002</mn><mo>,</mo><mn>540</mn></math></span></span><script id="MathJax-Element-78" type="math/tex">\$1,002,540</script> that was required to meet the same demand without the hydro plants.  The detailed plan for each time period is as follows.</p>
<h3 id="Unit-Commitments">Unit Commitments<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/electrical_power_generation_1_2/electrical_power2.html#Unit-Commitments">¶</a></h3><p>The following table shows the number of thermal generators of each type that are active in each time period in the optimal solution:</p>


In [ ]:

rows = ["Thermal" + str(t) for t in range(ntypes)]
units = pd.DataFrame(columns=range(nperiods), index=rows, data=0.0)

for t in range(ntypes):
    for p in range(nperiods):
        units.loc["Thermal"+str(t), p] = ngen[t,p].x
units




<p>The following shows the number of generators of each type that must be started in each time period to achieve this plan (recall that the model assumes that 5 are available at the beginning of the time horizon):</p>


In [ ]:

rows = ["HydroA", "HydroB"]
hydrotable = pd.DataFrame(columns=range(nperiods), index=rows, data=0.0)

for p in range(nperiods):
    hydrotable.loc["HydroA", p] = int(hydro[0,p].x)
    hydrotable.loc["HydroB", p] = int(hydro[1,p].x)
hydrotable




<p>The hydro plants are lightly used in the schedule - we only use plant B for the last two time periods.</p>
<p>The following shows the pumping that must be done in order to support this level of hydro activity</p>


In [ ]:

pumptable = pd.DataFrame(columns=range(nperiods), index=["Pumping"], data=0.0)

for p in range(nperiods):
        pumptable.loc["Pumping", p] = pumping[p].x
pumptable




<p>It is interesting to note that the plan simultaneously operates HydroB and pumps in time period 4.
While it might appear that costs could be reduced by operating the hydro plant at a lower load, in this model the hydro plants have fixed output.  If it makes economic sense to turn a hydro plant on, then we have to replenish the water drained, even if it means using some of the generated electricity to pump.</p>



<hr/>
<h2 id="References">References<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/electrical_power_generation_1_2/electrical_power2.html#References">¶</a></h2><p>H. Paul Williams, Model Building in Mathematical Programming, fifth edition.</p>
<p>Copyright © 2020 Gurobi Optimization, LLC</p>
